In [1]:
import os
import pickle
import string

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW09OT-MakeConcordance-1Samuel.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""


import os
import pickle
import string

from BibleWordList2 import *

BUILDAWIP = 1
BUILDNWIV = 1
BUILDPNIP = 1
BUILDNWIC = 1
BUILDNLIW = 1
BUILDBPIP = 1

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

FirstSamuel01 = [
    ['1Samuel','1Sa','01','001','1',"Now there was a certain man of Ramathaim-zophim, of mount Ephraim, and his name \emph{was} Elkanah, the son of Jeroham, the son of Elihu, the son of Tohu, the son of Zuph, an Ephrathite:"],
    ['1Samuel','1Sa','01','002','2',"And he had two wives; the name of the one \emph{was} Hannah, and the name of the other Peninnah: and Peninnah had children, but Hannah had no children."],
    ['1Samuel','1Sa','01','003','3',"And this man went up out of his city yearly to worship and to sacrifice unto the LORD of hosts in Shiloh. And the two sons of Eli, Hophni and Phinehas, the priests of the LORD, \emph{were} there."],
    ['1Samuel','1Sa','01','004','4',"And when the time was that Elkanah offered, he gave to Peninnah his wife, and to all her sons and her daughters, portions:"],
    ['1Samuel','1Sa','01','005','5',"But unto Hannah he gave a worthy portion; for he loved Hannah: but the LORD had shut up her womb."],
['1Samuel','1Sa','01','006','6',"And her adversary also provoked her sore, for to make her fret, because the LORD had shut up her womb."],
['1Samuel','1Sa','01','007','7',"And \emph{as} he did so year by year, when she went up to the house of the LORD, so she provoked her; therefore she wept, and did not eat."],
['1Samuel','1Sa','01','008','8',"Then said Elkanah her husband to her, Hannah, why weepest thou? and why eatest thou not? and why is thy heart grieved? \emph{am} not I better to thee than ten sons?"],
['1Samuel','1Sa','01','009','9',"So Hannah rose up after they had eaten in Shiloh, and after they had drunk. Now Eli the priest sat upon a seat by a post of the temple of the LORD."],
['1Samuel','1Sa','01','010','10',"And she \emph{was} in bitterness of soul, and prayed unto the LORD, and wept sore."],
['1Samuel','1Sa','01','011','11',"And she vowed a vow, and said, O LORD of hosts, if thou wilt indeed look on the affliction of thine handmaid, and remember me, and not forget thine handmaid, but wilt give unto thine handmaid a man child, then I will give him unto the LORD all the days of his life, and there shall no razor come upon his head."],
['1Samuel','1Sa','01','012','12',"And it came to pass, as she continued praying before the LORD, that Eli marked her mouth."],
['1Samuel','1Sa','01','013','13',"Now Hannah, she spake in her heart; only her lips moved, but her voice was not heard: therefore Eli thought she had been drunken."],
['1Samuel','1Sa','01','014','14',"And Eli said unto her, How long wilt thou be drunken? put away thy wine from thee."],
['1Samuel','1Sa','01','015','15',"And Hannah answered and said, No, my lord, I \emph{am} a woman of a sorrowful spirit: I have drunk neither wine nor strong drink, but have poured out my soul before the LORD."],
['1Samuel','1Sa','01','016','16',"Count not thine handmaid for a daughter of Belial: for out of the abundance of my complaint and grief have I spoken hitherto."],
['1Samuel','1Sa','01','017','17',"Then Eli answered and said, Go in peace: and the God of Israel grant \emph{thee} thy petition that thou hast asked of him."],
['1Samuel','1Sa','01','018','18',"And she said, Let thine handmaid find grace in thy sight. So the woman went her way, and did eat, and her countenance was no more \emph{sad}."],
['1Samuel','1Sa','01','019','19',"And they rose up in the morning early, and worshipped before the LORD, and returned, and came to their house to Ramah: and Elkanah knew Hannah his wife; and the LORD remembered her."],
['1Samuel','1Sa','01','020','20',"Wherefore it came to pass, when the time was come about after Hannah had conceived, that she bare a son, and called his name Samuel, \emph{saying}, Because I have asked him of the LORD."],
['1Samuel','1Sa','01','021','21',"And the man Elkanah, and all his house, went up to offer unto the LORD the yearly sacrifice, and his vow."],
['1Samuel','1Sa','01','022','22',"But Hannah went not up; for she said unto her husband, \emph{I} \emph{will} \emph{not} \emph{go} \emph{up} until the child be weaned, and \emph{then} I will bring him, that he may appear before the LORD, and there abide for ever."],
['1Samuel','1Sa','01','023','23',"And Elkanah her husband said unto her, Do what seemeth thee good; tarry until thou have weaned him; only the LORD establish his word. So the woman abode, and gave her son suck until she weaned him."],
['1Samuel','1Sa','01','024','24',"And when she had weaned him, she took him up with her, with three bullocks, and one ephah of flour, and a bottle of wine, and brought him unto the house of the LORD in Shiloh: and the child \emph{was} young."],
['1Samuel','1Sa','01','025','25',"And they slew a bullock, and brought the child to Eli."],
    ['1Samuel','1Sa','01','026','26',"And she said, Oh my lord, \emph{as} thy soul liveth, my lord, I \emph{am} the woman that stood by thee here, praying unto the LORD."],
    ['1Samuel','1Sa','01','027','27',"For this child I prayed; and the LORD hath given me my petition which I asked of him:"],
    ['1Samuel','1Sa','01','028','28',"Therefore also I have lent him to the LORD; as long as he liveth he shall be lent to the LORD. And he worshipped the LORD there."]
]

FirstSamuel02 = [
    ['1Samuel','1Sa','02','001','1',"And Hannah prayed, and said, My heart rejoiceth in the LORD, mine horn is exalted in the LORD: my mouth is enlarged over mine enemies; because I rejoice in thy salvation."],
['1Samuel','1Sa','02','002','2',"\emph{There} \emph{is} none holy as the LORD: for \emph{there} \emph{is} none beside thee: neither \emph{is} \emph{there} any rock like our God."],
['1Samuel','1Sa','02','003','3',"Talk no more so exceeding proudly; let \emph{not} arrogancy come out of your mouth: for the LORD \emph{is} a God of knowledge, and by him actions are weighed."],
['1Samuel','1Sa','02','004','4',"The bows of the mighty men \emph{are} broken, and they that stumbled are girded with strength."],
['1Samuel','1Sa','02','005','5',"\emph{They} \emph{that} \emph{were} full have hired out themselves for bread; and \emph{they} \emph{that} \emph{were} hungry ceased: so that the barren hath born seven; and she that hath many children is waxed feeble."],
['1Samuel','1Sa','02','006','6',"The LORD killeth, and maketh alive: he bringeth down to the grave, and bringeth up."],
['1Samuel','1Sa','02','007','7',"The LORD maketh poor, and maketh rich: he bringeth low, and lifteth up."],
['1Samuel','1Sa','02','008','8',"He raiseth up the poor out of the dust, \emph{and} lifteth up the beggar from the dunghill, to set \emph{them} among princes, and to make them inherit the throne of glory: for the pillars of the earth \emph{are} the LORD'S, and he hath set the world upon them."],
['1Samuel','1Sa','02','009','9',"He will keep the feet of his saints, and the wicked shall be silent in darkness; for by strength shall no man prevail."],
['1Samuel','1Sa','02','010','10',"The adversaries of the LORD shall be broken to pieces; out of heaven shall he thunder upon them: the LORD shall judge the ends of the earth; and he shall give strength unto his king, and exalt the horn of his anointed."],
['1Samuel','1Sa','02','011','11',"And Elkanah went to Ramah to his house. And the child did minister unto the LORD before Eli the priest."],
['1Samuel','1Sa','02','012','12',"Now the sons of Eli \emph{were} sons of Belial; they knew not the LORD."],
['1Samuel','1Sa','02','013','13',"And the priests' custom with the people \emph{was,} \emph{that}, when any man offered sacrifice, the priest's servant came, while the flesh was in seething, with a fleshhook of three teeth in his hand;"],
['1Samuel','1Sa','02','014','14',"And he struck \emph{it} into the pan, or kettle, or caldron, or pot; all that the fleshhook brought up the priest took for himself. So they did in Shiloh unto all the Israelites that came thither."],
['1Samuel','1Sa','02','015','15',"Also before they burnt the fat, the priest's servant came, and said to the man that sacrificed, Give flesh to roast for the priest; for he will not have sodden flesh of thee, but raw."],
['1Samuel','1Sa','02','016','16',"And \emph{if} any man said unto him, Let them not fail to burn the fat presently, and \emph{then} take \emph{as} \emph{much} as thy soul desireth; then he would answer him, \emph{Nay}; but thou shalt give \emph{it} \emph{me} now: and if not, I will take \emph{it} by force."],
['1Samuel','1Sa','02','017','17',"Wherefore the sin of the young men was very great before the LORD: for men abhorred the offering of the LORD."],
['1Samuel','1Sa','02','018','18',"But Samuel ministered before the LORD, \emph{being} a child, girded with a linen ephod."],
['1Samuel','1Sa','02','019','19',"Moreover his mother made him a little coat, and brought \emph{it} to him from year to year, when she came up with her husband to offer the yearly sacrifice."],
['1Samuel','1Sa','02','020','20',"And Eli blessed Elkanah and his wife, and said, The LORD give thee seed of this woman for the loan which is lent to the LORD. And they went unto their own home."],
['1Samuel','1Sa','02','021','21',"And the LORD visited Hannah, so that she conceived, and bare three sons and two daughters. And the child Samuel grew before the LORD."],
['1Samuel','1Sa','02','022','22',"Now Eli was very old, and heard all that his sons did unto all Israel; and how they lay with the women that assembled \emph{at} the door of the tabernacle of the congregation."],
['1Samuel','1Sa','02','023','23',"And he said unto them, Why do ye such things? for I hear of your evil dealings by all this people."],
['1Samuel','1Sa','02','024','24',"Nay, my sons; for \emph{it} \emph{is} no good report that I hear: ye make the LORD'S people to transgress."],
['1Samuel','1Sa','02','025','25',"If one man sin against another, the judge shall judge him: but if a man sin against the LORD, who shall intreat for him? Notwithstanding they hearkened not unto the voice of their father, because the LORD would slay them."],
['1Samuel','1Sa','02','026','26',"And the child Samuel grew on, and was in favour both with the LORD, and also with men."],
['1Samuel','1Sa','02','027','27',"And there came a man of God unto Eli, and said unto him, Thus saith the LORD, Did I plainly appear unto the house of thy father, when they were in Egypt in Pharaoh's house?"],
['1Samuel','1Sa','02','028','28',"And did I choose him out of all the tribes of Israel \emph{to} \emph{be} my priest, to offer upon mine altar, to burn incense, to wear an ephod before me? and did I give unto the house of thy father all the offerings made by fire of the children of Israel?"],
['1Samuel','1Sa','02','029','29',"Wherefore kick ye at my sacrifice and at mine offering, which I have commanded \emph{in} \emph{my} habitation; and honourest thy sons above me, to make yourselves fat with the chiefest of all the offerings of Israel my people?"],
['1Samuel','1Sa','02','030','30',"Wherefore the LORD God of Israel saith, I said indeed \emph{that} thy house, and the house of thy father, should walk before me for ever: but now the LORD saith, Be it far from me; for them that honour me I will honour, and they that despise me shall be lightly esteemed."],
['1Samuel','1Sa','02','031','31',"Behold, the days come, that I will cut off thine arm, and the arm of thy father's house, that there shall not be an old man in thine house."],
['1Samuel','1Sa','02','032','32',"And thou shalt see an enemy \emph{in} \emph{my} habitation, in all \emph{the} \emph{wealth} which \emph{God} shall give Israel: and there shall not be an old man in thine house for ever."],
['1Samuel','1Sa','02','033','33',"And the man of thine, \emph{whom} I shall not cut off from mine altar, \emph{shall} \emph{be} to consume thine eyes, and to grieve thine heart: and all the increase of thine house shall die in the flower of their age."],
['1Samuel','1Sa','02','034','34',"And this \emph{shall} \emph{be} a sign unto thee, that shall come upon thy two sons, on Hophni and Phinehas; in one day they shall die both of them."],
['1Samuel','1Sa','02','035','35',"And I will raise me up a faithful priest, \emph{that} shall do according to \emph{that} which \emph{is} in mine heart and in my mind: and I will build him a sure house; and he shall walk before mine anointed for ever."],
['1Samuel','1Sa','02','036','36',"And it shall come to pass, \emph{that} every one that is left in thine house shall come \emph{and} crouch to him for a piece of silver and a morsel of bread, and shall say, Put me, I pray thee, into one of the priests' offices, that I may eat a piece of bread."]]

FirstSamuel03 = [
    ['1Samuel','1Sa','03','001','1',"And the child Samuel ministered unto the LORD before Eli. And the word of the LORD was precious in those days; \emph{there} \emph{was} no open vision."],
['1Samuel','1Sa','03','002','2',"And it came to pass at that time, when Eli \emph{was} laid down in his place, and his eyes began to wax dim, \emph{that} he could not see;"],
['1Samuel','1Sa','03','003','3',"And ere the lamp of God went out in the temple of the LORD, where the ark of God \emph{was}, and Samuel was laid down \emph{to} \emph{sleep};"],
['1Samuel','1Sa','03','004','4',"That the LORD called Samuel: and he answered, Here \emph{am} I."],
['1Samuel','1Sa','03','005','5',"And he ran unto Eli, and said, Here \emph{am} I; for thou calledst me. And he said, I called not; lie down again. And he went and lay down."],
['1Samuel','1Sa','03','006','6',"And the LORD called yet again, Samuel. And Samuel arose and went to Eli, and said, Here \emph{am} I; for thou didst call me. And he answered, I called not, my son; lie down again."],
['1Samuel','1Sa','03','007','7',"Now Samuel did not yet know the LORD, neither was the word of the LORD yet revealed unto him."],
['1Samuel','1Sa','03','008','8',"And the LORD called Samuel again the third time. And he arose and went to Eli, and said, Here \emph{am} I; for thou didst call me. And Eli perceived that the LORD had called the child."],
['1Samuel','1Sa','03','009','9',"Therefore Eli said unto Samuel, Go, lie down: and it shall be, if he call thee, that thou shalt say, Speak, LORD; for thy servant heareth. So Samuel went and lay down in his place."],
['1Samuel','1Sa','03','010','10',"And the LORD came, and stood, and called as at other times, Samuel, Samuel. Then Samuel answered, Speak; for thy servant heareth."],
['1Samuel','1Sa','03','011','11',"And the LORD said to Samuel, Behold, I will do a thing in Israel, at which both the ears of every one that heareth it shall tingle."],
['1Samuel','1Sa','03','012','12',"In that day I will perform against Eli all \emph{things} which I have spoken concerning his house: when I begin, I will also make an end."],
['1Samuel','1Sa','03','013','13',"For I have told him that I will judge his house for ever for the iniquity which he knoweth; because his sons made themselves vile, and he restrained them not."],
['1Samuel','1Sa','03','014','14',"And therefore I have sworn unto the house of Eli, that the iniquity of Eli's house shall not be purged with sacrifice nor offering for ever."],
['1Samuel','1Sa','03','015','15',"And Samuel lay until the morning, and opened the doors of the house of the LORD. And Samuel feared to shew Eli the vision."],
['1Samuel','1Sa','03','016','16',"Then Eli called Samuel, and said, Samuel, my son. And he answered, Here \emph{am} I."],
['1Samuel','1Sa','03','017','17',"And he said, What \emph{is} the thing that \emph{the} \emph{LORD} hath said unto thee? I pray thee hide \emph{it} not from me: God do so to thee, and more also, if thou hide \emph{any} thing from me of all the things that he said unto thee."],
['1Samuel','1Sa','03','018','18',"And Samuel told him every whit, and hid nothing from him. And he said, It \emph{is} the LORD: let him do what seemeth him good."],
['1Samuel','1Sa','03','019','19',"And Samuel grew, and the LORD was with him, and did let none of his words fall to the ground."],
['1Samuel','1Sa','03','020','20',"And all Israel from Dan even to Beer-sheba knew that Samuel \emph{was} established \emph{to} \emph{be} a prophet of the LORD."],
['1Samuel','1Sa','03','021','21',"And the LORD appeared again in Shiloh: for the LORD revealed himself to Samuel in Shiloh by the word of the LORD."]
]

FirstSamuel04 = [
    ['1Samuel','1Sa','04','001','1',"And the word of Samuel came to all Israel. Now Israel went out against the Philistines to battle, and pitched beside Eben-ezer: and the Philistines pitched in Aphek."],
    ['1Samuel','1Sa','04','002','2',"And the Philistines put themselves in array against Israel: and when they joined battle, Israel was smitten before the Philistines: and they slew of the army in the field about four thousand men."],
    ['1Samuel','1Sa','04','003','3',"And when the people were come into the camp, the elders of Israel said, Wherefore hath the LORD smitten us to day before the Philistines? Let us fetch the ark of the covenant of the LORD out of Shiloh unto us, that, when it cometh among us, it may save us out of the hand of our enemies."],
    ['1Samuel','1Sa','04','004','4',"So the people sent to Shiloh, that they might bring from thence the ark of the covenant of the LORD of hosts, which dwelleth \emph{between} the cherubims: and the two sons of Eli, Hophni and Phinehas, \emph{were} there with the ark of the covenant of God."],
    ['1Samuel','1Sa','04','005','5',"And when the ark of the covenant of the LORD came into the camp, all Israel shouted with a great shout, so that the earth rang again."],
    ['1Samuel','1Sa','04','006','6',"And when the Philistines heard the noise of the shout, they said, What \emph{meaneth} the noise of this great shout in the camp of the Hebrews? And they understood that the ark of the LORD was come into the camp."],
    ['1Samuel','1Sa','04','007','7',"And the Philistines were afraid, for they said, God is come into the camp. And they said, Woe unto us! for there hath not been such a thing heretofore."],
    ['1Samuel','1Sa','04','008','8',"Woe unto us! who shall deliver us out of the hand of these mighty Gods? these \emph{are} the Gods that smote the Egyptians with all the plagues in the wilderness."],
    ['1Samuel','1Sa','04','009','9',"Be strong, and quit yourselves like men, O ye Philistines, that ye be not servants unto the Hebrews, as they have been to you: quit yourselves like men, and fight."],
    ['1Samuel','1Sa','04','010','10',"And the Philistines fought, and Israel was smitten, and they fled every man into his tent: and there was a very great slaughter; for there fell of Israel thirty thousand footmen."],
    ['1Samuel','1Sa','04','011','11',"And the ark of God was taken; and the two sons of Eli, Hophni and Phinehas, were slain."],
    ['1Samuel','1Sa','04','012','12',"And there ran a man of Benjamin out of the army, and came to Shiloh the same day with his clothes rent, and with earth upon his head."],
    ['1Samuel','1Sa','04','013','13',"And when he came, lo, Eli sat upon a seat by the wayside watching: for his heart trembled for the ark of God. And when the man came into the city, and told \emph{it}, all the city cried out."],
    ['1Samuel','1Sa','04','014','14',"And when Eli heard the noise of the crying, he said, What \emph{meaneth} the noise of this tumult? And the man came in hastily, and told Eli."],
    ['1Samuel','1Sa','04','015','15',"Now Eli was ninety and eight years old; and his eyes were dim, that he could not see."],
    ['1Samuel','1Sa','04','016','16',"And the man said unto Eli, I \emph{am} he that came out of the army, and I fled to day out of the army. And he said, What is there done, my son?"],
    ['1Samuel','1Sa','04','017','17',"And the messenger answered and said, Israel is fled before the Philistines, and there hath been also a great slaughter among the people, and thy two sons also, Hophni and Phinehas, are dead, and the ark of God is taken."],
    ['1Samuel','1Sa','04','018','18',"And it came to pass, when he made mention of the ark of God, that he fell from off the seat backward by the side of the gate, and his neck brake, and he died: for he was an old man, and heavy. And he had judged Israel forty years."],
    ['1Samuel','1Sa','04','019','19',"And his daughter in law, Phinehas' wife, was with child, \emph{near} to be delivered: and when she heard the tidings that the ark of God was taken, and that her father in law and her husband were dead, she bowed herself and travailed; for her pains came upon her."],
    ['1Samuel','1Sa','04','020','20',"And about the time of her death the women that stood by her said unto her, Fear not; for thou hast born a son. But she answered not, neither did she regard \emph{it}."],
    ['1Samuel','1Sa','04','021','21',"And she named the child I-chabod, saying, The glory is departed from Israel: because the ark of God was taken, and because of her father in law and her husband."],
    ['1Samuel','1Sa','04','022','22',"And she said, The glory is departed from Israel: for the ark of God is taken."]
]

FirstSamuel05 = [
    ['1Samuel','1Sa','05','001','1',"And the Philistines took the ark of God, and brought it from Eben-ezer unto Ashdod."],
    ['1Samuel','1Sa','05','002','2',"When the Philistines took the ark of God, they brought it into the house of Dagon, and set it by Dagon."],
    ['1Samuel','1Sa','05','003','3',"And when they of Ashdod arose early on the morrow, behold, Dagon \emph{was} fallen upon his face to the earth before the ark of the LORD. And they took Dagon, and set him in his place again."],
    ['1Samuel','1Sa','05','004','4',"And when they arose early on the morrow morning, behold, Dagon \emph{was} fallen upon his face to the ground before the ark of the LORD; and the head of Dagon and both the palms of his hands \emph{were} cut off upon the threshold; only \emph{the} \emph{stump} \emph{of} Dagon was left to him."],
    ['1Samuel','1Sa','05','005','5',"Therefore neither the priests of Dagon, nor any that come into Dagon's house, tread on the threshold of Dagon in Ashdod unto this day."],
    ['1Samuel','1Sa','05','006','6',"But the hand of the LORD was heavy upon them of Ashdod, and he destroyed them, and smote them with emerods, \emph{even} Ashdod and the coasts thereof."],
    ['1Samuel','1Sa','05','007','7',"And when the men of Ashdod saw that \emph{it} \emph{was} so, they said, The ark of the God of Israel shall not abide with us: for his hand is sore upon us, and upon Dagon our god."],
    ['1Samuel','1Sa','05','008','8',"They sent therefore and gathered all the lords of the Philistines unto them, and said, What shall we do with the ark of the God of Israel? And they answered, Let the ark of the God of Israel be carried about unto Gath. And they carried the ark of the God of Israel about \emph{thither}."],
    ['1Samuel','1Sa','05','009','9',"And it was \emph{so}, that, after they had carried it about, the hand of the LORD was against the city with a very great destruction: and he smote the men of the city, both small and great, and they had emerods in their secret parts."],
    ['1Samuel','1Sa','05','010','10',"Therefore they sent the ark of God to Ekron. And it came to pass, as the ark of God came to Ekron, that the Ekronites cried out, saying, They have brought about the ark of the God of Israel to us, to slay us and our people."],
    ['1Samuel','1Sa','05','011','11',"So they sent and gathered together all the lords of the Philistines, and said, Send away the ark of the God of Israel, and let it go again to his own place, that it slay us not, and our people: for there was a deadly destruction throughout all the city; the hand of God was very heavy there."],
    ['1Samuel','1Sa','05','012','12',"And the men that died not were smitten with the emerods: and the cry of the city went up to heaven."]
]


FirstSamuel06 = [
    ['1Samuel','1Sa','06','001','1',"And the ark of the LORD was in the country of the Philistines seven months."],
    ['1Samuel','1Sa','06','002','2',"And the Philistines called for the priests and the diviners, saying, What shall we do to the ark of the LORD? tell us wherewith we shall send it to his place."],
    ['1Samuel','1Sa','06','003','3',"And they said, If ye send away the ark of the God of Israel, send it not empty; but in any wise return him a trespass offering: then ye shall be healed, and it shall be known to you why his hand is not removed from you."],
    ['1Samuel','1Sa','06','004','4',"Then said they, What \emph{shall} \emph{be} the trespass offering which we shall return to him? They answered, Five golden emerods, and five golden mice, \emph{according} \emph{to} the number of the lords of the Philistines: for one plague \emph{was} on you all, and on your lords."],
    ['1Samuel','1Sa','06','005','5',"Wherefore ye shall make images of your emerods, and images of your mice that mar the land; and ye shall give glory unto the God of Israel: peradventure he will lighten his hand from off you, and from off your gods, and from off your land."],
    ['1Samuel','1Sa','06','006','6',"Wherefore then do ye harden your hearts, as the Egyptians and Pharaoh hardened their hearts? when he had wrought wonderfully among them, did they not let the people go, and they departed?"],
    ['1Samuel','1Sa','06','007','7',"Now therefore make a new cart, and take two milch kine, on which there hath come no yoke, and tie the kine to the cart, and bring their calves home from them:"],
    ['1Samuel','1Sa','06','008','8',"And take the ark of the LORD, and lay it upon the cart; and put the jewels of gold, which ye return him \emph{for} a trespass offering, in a coffer by the side thereof; and send it away, that it may go."],
    ['1Samuel','1Sa','06','009','9',"And see, if it goeth up by the way of his own coast to Beth-shemesh, \emph{then} he hath done us this great evil: but if not, then we shall know that \emph{it} \emph{is} not his hand \emph{that} smote us: it \emph{was} a chance \emph{that} happened to us."],
    ['1Samuel','1Sa','06','010','10',"And the men did so; and took two milch kine, and tied them to the cart, and shut up their calves at home:"],
    ['1Samuel','1Sa','06','011','11',"And they laid the ark of the LORD upon the cart, and the coffer with the mice of gold and the images of their emerods."],
    ['1Samuel','1Sa','06','012','12',"And the kine took the straight way to the way of Beth-shemesh, \emph{and} went along the highway, lowing as they went, and turned not aside \emph{to} the right hand or \emph{to} the left; and the lords of the Philistines went after them unto the border of Beth-shemesh."],
    ['1Samuel','1Sa','06','013','13',"And \emph{they} \emph{of} Beth-shemesh \emph{were} reaping their wheat harvest in the valley: and they lifted up their eyes, and saw the ark, and rejoiced to see \emph{it}."],
    ['1Samuel','1Sa','06','014','14',"And the cart came into the field of Joshua, a Beth-shemite, and stood there, where \emph{there} \emph{was} a great stone: and they clave the wood of the cart, and offered the kine a burnt offering unto the LORD."],
    ['1Samuel','1Sa','06','015','15',"And the Levites took down the ark of the LORD, and the coffer that \emph{was} with it, wherein the jewels of gold \emph{were}, and put \emph{them} on the great stone: and the men of Beth-shemesh offered burnt offerings and sacrificed sacrifices the same day unto the LORD."],
    ['1Samuel','1Sa','06','016','16',"And when the five lords of the Philistines had seen \emph{it}, they returned to Ekron the same day."],
    ['1Samuel','1Sa','06','017','17',"And these \emph{are} the golden emerods which the Philistines returned \emph{for} a trespass offering unto the LORD; for Ashdod one, for Gaza one, for Askelon one, for Gath one, for Ekron one;"],
    ['1Samuel','1Sa','06','018','18',"And the golden mice, \emph{according} \emph{to} the number of all the cities of the Philistines \emph{belonging} to the five lords, \emph{both} of fenced cities, and of country villages, even unto the great \emph{stone} \emph{of} Abel, whereon they set down the ark of the LORD: \emph{which} \emph{stone} \emph{remaineth} unto this day in the field of Joshua, the Beth-shemite."],
    ['1Samuel','1Sa','06','019','19',"And he smote the men of Beth-shemesh, because they had looked into the ark of the LORD, even he smote of the people fifty thousand and threescore and ten men: and the people lamented, because the LORD had smitten \emph{many} of the people with a great slaughter."],
    ['1Samuel','1Sa','06','020','20',"And the men of Beth-shemesh said, Who is able to stand before this holy LORD God? and to whom shall he go up from us?"],
    ['1Samuel','1Sa','06','021','21',"And they sent messengers to the inhabitants of Kirjath-jearim, saying, The Philistines have brought again the ark of the LORD; come ye down, \emph{and} fetch it up to you."]
]

FirstSamuel07 = [
    ['1Samuel','1Sa','07','001','1',"And the men of Kirjath-jearim came, and fetched up the ark of the LORD, and brought it into the house of Abinadab in the hill, and sanctified Eleazar his son to keep the ark of the LORD."],
['1Samuel','1Sa','07','002','2',"And it came to pass, while the ark abode in Kirjath-jearim, that the time was long; for it was twenty years: and all the house of Israel lamented after the LORD."],
['1Samuel','1Sa','07','003','3',"And Samuel spake unto all the house of Israel, saying, If ye do return unto the LORD with all your hearts, \emph{then} put away the strange gods and Ashtaroth from among you, and prepare your hearts unto the LORD, and serve him only: and he will deliver you out of the hand of the Philistines."],
['1Samuel','1Sa','07','004','4',"Then the children of Israel did put away Baalim and Ashtaroth, and served the LORD only."],
['1Samuel','1Sa','07','005','5',"And Samuel said, Gather all Israel to Mizpeh, and I will pray for you unto the LORD."],
['1Samuel','1Sa','07','006','6',"And they gathered together to Mizpeh, and drew water, and poured \emph{it} out before the LORD, and fasted on that day, and said there, We have sinned against the LORD. And Samuel judged the children of Israel in Mizpeh."],
['1Samuel','1Sa','07','007','7',"And when the Philistines heard that the children of Israel were gathered together to Mizpeh, the lords of the Philistines went up against Israel. And when the children of Israel heard \emph{it}, they were afraid of the Philistines."],
['1Samuel','1Sa','07','008','8',"And the children of Israel said to Samuel, Cease not to cry unto the LORD our God for us, that he will save us out of the hand of the Philistines."],
['1Samuel','1Sa','07','009','9',"And Samuel took a sucking lamb, and offered \emph{it} \emph{for} a burnt offering wholly unto the LORD: and Samuel cried unto the LORD for Israel; and the LORD heard him."],
['1Samuel','1Sa','07','010','10',"And as Samuel was offering up the burnt offering, the Philistines drew near to battle against Israel: but the LORD thundered with a great thunder on that day upon the Philistines, and discomfited them; and they were smitten before Israel."],
['1Samuel','1Sa','07','011','11',"And the men of Israel went out of Mizpeh, and pursued the Philistines, and smote them, until \emph{they} \emph{came} under Beth-car."],
['1Samuel','1Sa','07','012','12',"Then Samuel took a stone, and set \emph{it} between Mizpeh and Shen, and called the name of it Eben-ezer, saying, Hitherto hath the LORD helped us."],
['1Samuel','1Sa','07','013','13',"So the Philistines were subdued, and they came no more into the coast of Israel: and the hand of the LORD was against the Philistines all the days of Samuel."],
['1Samuel','1Sa','07','014','14',"And the cities which the Philistines had taken from Israel were restored to Israel, from Ekron even unto Gath; and the coasts thereof did Israel deliver out of the hands of the Philistines. And there was peace between Israel and the Amorites."],
['1Samuel','1Sa','07','015','15',"And Samuel judged Israel all the days of his life."],
['1Samuel','1Sa','07','016','16',"And he went from year to year in circuit to Beth-el, and Gilgal, and Mizpeh, and judged Israel in all those places."],
['1Samuel','1Sa','07','017','17',"And his return \emph{was} to Ramah; for there \emph{was} his house; and there he judged Israel; and there he built an altar unto the LORD."]
]

FirstSamuel08 = [
    ['1Samuel','1Sa','08','001','1',"And it came to pass, when Samuel was old, that he made his sons judges over Israel."],
['1Samuel','1Sa','08','002','2',"Now the name of his firstborn was Joel; and the name of his second, Abiah: \emph{they} \emph{were} judges in Beer-sheba."],
['1Samuel','1Sa','08','003','3',"And his sons walked not in his ways, but turned aside after lucre, and took bribes, and perverted judgment."],
['1Samuel','1Sa','08','004','4',"Then all the elders of Israel gathered themselves together, and came to Samuel unto Ramah,"],
['1Samuel','1Sa','08','005','5',"And said unto him, Behold, thou art old, and thy sons walk not in thy ways: now make us a king to judge us like all the nations."],
['1Samuel','1Sa','08','006','6',"But the thing displeased Samuel, when they said, Give us a king to judge us. And Samuel prayed unto the LORD."],
['1Samuel','1Sa','08','007','7',"And the LORD said unto Samuel, Hearken unto the voice of the people in all that they say unto thee: for they have not rejected thee, but they have rejected me, that I should not reign over them."],
['1Samuel','1Sa','08','008','8',"According to all the works which they have done since the day that I brought them up out of Egypt even unto this day, wherewith they have forsaken me, and served other gods, so do they also unto thee."],
['1Samuel','1Sa','08','009','9',"Now therefore hearken unto their voice: howbeit yet protest solemnly unto them, and shew them the manner of the king that shall reign over them."],
['1Samuel','1Sa','08','010','10',"And Samuel told all the words of the LORD unto the people that asked of him a king."],
['1Samuel','1Sa','08','011','11',"And he said, This will be the manner of the king that shall reign over you: He will take your sons, and appoint \emph{them} for himself, for his chariots, and \emph{to} \emph{be} his horsemen; and \emph{some} shall run before his chariots."],
['1Samuel','1Sa','08','012','12',"And he will appoint him captains over thousands, and captains over fifties; and \emph{will} \emph{set} \emph{them} to ear his ground, and to reap his harvest, and to make his instruments of war, and instruments of his chariots."],
['1Samuel','1Sa','08','013','13',"And he will take your daughters \emph{to} \emph{be} confectionaries, and \emph{to} \emph{be} cooks, and \emph{to} \emph{be} bakers."],
['1Samuel','1Sa','08','014','14',"And he will take your fields, and your vineyards, and your oliveyards, \emph{even} the best \emph{of} \emph{them}, and give \emph{them} to his servants."],
['1Samuel','1Sa','08','015','15',"And he will take the tenth of your seed, and of your vineyards, and give to his officers, and to his servants."],
['1Samuel','1Sa','08','016','16',"And he will take your menservants, and your maidservants, and your goodliest young men, and your asses, and put \emph{them} to his work."],
['1Samuel','1Sa','08','017','17',"He will take the tenth of your sheep: and ye shall be his servants."],
['1Samuel','1Sa','08','018','18',"And ye shall cry out in that day because of your king which ye shall have chosen you; and the LORD will not hear you in that day."],
['1Samuel','1Sa','08','019','19',"Nevertheless the people refused to obey the voice of Samuel; and they said, Nay; but we will have a king over us;"],
['1Samuel','1Sa','08','020','20',"That we also may be like all the nations; and that our king may judge us, and go out before us, and fight our battles."],
['1Samuel','1Sa','08','021','21',"And Samuel heard all the words of the people, and he rehearsed them in the ears of the LORD."],
['1Samuel','1Sa','08','022','22',"And the LORD said to Samuel, Hearken unto their voice, and make them a king. And Samuel said unto the men of Israel, Go ye every man unto his city."]
]

FirstSamuel09 = [
    ['1Samuel','1Sa','09','001','1',"Now there was a man of Benjamin, whose name \emph{was} Kish, the son of Abiel, the son of Zeror, the son of Bechorath, the son of Aphiah, a Benjamite, a mighty man of power."],
['1Samuel','1Sa','09','002','2',"And he had a son, whose name \emph{was} Saul, a choice young man, and a goodly: and \emph{there} \emph{was} not among the children of Israel a goodlier person than he: from his shoulders and upward \emph{he} \emph{was} higher than any of the people."],
['1Samuel','1Sa','09','003','3',"And the asses of Kish Saul's father were lost. And Kish said to Saul his son, Take now one of the servants with thee, and arise, go seek the asses."],
['1Samuel','1Sa','09','004','4',"And he passed through mount Ephraim, and passed through the land of Shalisha, but they found \emph{them} not: then they passed through the land of Shalim, and \emph{there} \emph{they} \emph{were} not: and he passed through the land of the Benjamites, but they found \emph{them} not."],
['1Samuel','1Sa','09','005','5',"\emph{And} when they were come to the land of Zuph, Saul said to his servant that \emph{was} with him, Come, and let us return; lest my father leave \emph{caring} for the asses, and take thought for us."],
['1Samuel','1Sa','09','006','6',"And he said unto him, Behold now, \emph{there} \emph{is} in this city a man of God, and \emph{he} \emph{is} an honourable man; all that he saith cometh surely to pass: now let us go thither; peradventure he can shew us our way that we should go."],
['1Samuel','1Sa','09','007','7',"Then said Saul to his servant, But, behold, \emph{if} we go, what shall we bring the man? for the bread is spent in our vessels, and \emph{there} \emph{is} not a present to bring to the man of God: what have we?"],
['1Samuel','1Sa','09','008','8',"And the servant answered Saul again, and said, Behold, I have here at hand the fourth part of a shekel of silver: \emph{that} will I give to the man of God, to tell us our way."],
['1Samuel','1Sa','09','009','9',"(Beforetime in Israel, when a man went to enquire of God, thus he spake, Come, and let us go to the seer: for \emph{he} \emph{that} \emph{is} now \emph{called} a Prophet was beforetime called a Seer.)"],
['1Samuel','1Sa','09','010','10',"Then said Saul to his servant, Well said; come, let us go. So they went unto the city where the man of God \emph{was}."],
['1Samuel','1Sa','09','011','11',"\emph{And} as they went up the hill to the city, they found young maidens going out to draw water, and said unto them, Is the seer here?"],
['1Samuel','1Sa','09','012','12',"And they answered them, and said, He is; behold, \emph{he} \emph{is} before you: make haste now, for he came to day to the city; for \emph{there} \emph{is} a sacrifice of the people to day in the high place:"],
['1Samuel','1Sa','09','013','13',"As soon as ye be come into the city, ye shall straightway find him, before he go up to the high place to eat: for the people will not eat until he come, because he doth bless the sacrifice; \emph{and} afterwards they eat that be bidden. Now therefore get you up; for about this time ye shall find him."],
['1Samuel','1Sa','09','014','14',"And they went up into the city: \emph{and} when they were come into the city, behold, Samuel came out against them, for to go up to the high place."],
['1Samuel','1Sa','09','015','15',"Now the LORD had told Samuel in his ear a day before Saul came, saying,"],
['1Samuel','1Sa','09','016','16',"To morrow about this time I will send thee a man out of the land of Benjamin, and thou shalt anoint him \emph{to} \emph{be} captain over my people Israel, that he may save my people out of the hand of the Philistines: for I have looked upon my people, because their cry is come unto me."],
['1Samuel','1Sa','09','017','17',"And when Samuel saw Saul, the LORD said unto him, Behold the man whom I spake to thee of! this same shall reign over my people."],
['1Samuel','1Sa','09','018','18',"Then Saul drew near to Samuel in the gate, and said, Tell me, I pray thee, where the seer's house \emph{is}."],
['1Samuel','1Sa','09','019','19',"And Samuel answered Saul, and said, I \emph{am} the seer: go up before me unto the high place; for ye shall eat with me to day, and to morrow I will let thee go, and will tell thee all that \emph{is} in thine heart."],
['1Samuel','1Sa','09','020','20',"And as for thine asses that were lost three days ago, set not thy mind on them; for they are found. And on whom \emph{is} all the desire of Israel? \emph{Is} \emph{it} not on thee, and on all thy father's house?"],
['1Samuel','1Sa','09','021','21',"And Saul answered and said, \emph{Am} not I a Benjamite, of the smallest of the tribes of Israel? and my family the least of all the families of the tribe of Benjamin? wherefore then speakest thou so to me?"],
['1Samuel','1Sa','09','022','22',"And Samuel took Saul and his servant, and brought them into the parlour, and made them sit in the chiefest place among them that were bidden, which \emph{were} about thirty persons."],
['1Samuel','1Sa','09','023','23',"And Samuel said unto the cook, Bring the portion which I gave thee, of which I said unto thee, Set it by thee."],
['1Samuel','1Sa','09','024','24',"And the cook took up the shoulder, and \emph{that} which \emph{was} upon it, and set \emph{it} before Saul. And \emph{Samuel} said, Behold that which is left! set \emph{it} before thee, \emph{and} eat: for unto this time hath it been kept for thee since I said, I have invited the people. So Saul did eat with Samuel that day."],
['1Samuel','1Sa','09','025','25',"And when they were come down from the high place into the city, \emph{Samuel} communed with Saul upon the top of the house."],
    ['1Samuel','1Sa','09','026','26',"And they arose early: and it came to pass about the spring of the day, that Samuel called Saul to the top of the house, saying, Up, that I may send thee away. And Saul arose, and they went out both of them, he and Samuel, abroad."],
    ['1Samuel','1Sa','09','027','27',"\emph{And} as they were going down to the end of the city, Samuel said to Saul, Bid the servant pass on before us, (and he passed on,) but stand thou still a while, that I may shew thee the word of God."]
]

FirstSamuel10 = [
    ['1Samuel','1Sa','10','001','1',"Then Samuel took a vial of oil, and poured \emph{it} upon his head, and kissed him, and said, \emph{Is} \emph{it} not because the LORD hath anointed thee \emph{to} \emph{be} captain over his inheritance?"],
    ['1Samuel','1Sa','10','002','2',"When thou art departed from me to day, then thou shalt find two men by Rachel's sepulchre in the border of Benjamin at Zelzah; and they will say unto thee, The asses which thou wentest to seek are found: and, lo, thy father hath left the care of the asses, and sorroweth for you, saying, What shall I do for my son?"],
    ['1Samuel','1Sa','10','003','3',"Then shalt thou go on forward from thence, and thou shalt come to the plain of Tabor, and there shall meet thee three men going up to God to Beth-el, one carrying three kids, and another carrying three loaves of bread, and another carrying a bottle of wine:"],
    ['1Samuel','1Sa','10','004','4',"And they will salute thee, and give thee two \emph{loaves} of bread; which thou shalt receive of their hands."],
    ['1Samuel','1Sa','10','005','5',"After that thou shalt come to the hill of God, where \emph{is} the garrison of the Philistines: and it shall come to pass, when thou art come thither to the city, that thou shalt meet a company of prophets coming down from the high place with a psaltery, and a tabret, and a pipe, and a harp, before them; and they shall prophesy:"],
    ['1Samuel','1Sa','10','006','6',"And the Spirit of the LORD will come upon thee, and thou shalt prophesy with them, and shalt be turned into another man."],
    ['1Samuel','1Sa','10','007','7',"And let it be, when these signs are come unto thee, \emph{that} thou do as occasion serve thee; for God \emph{is} with thee."],
    ['1Samuel','1Sa','10','008','8',"And thou shalt go down before me to Gilgal; and, behold, I will come down unto thee, to offer burnt offerings, \emph{and} to sacrifice sacrifices of peace offerings: seven days shalt thou tarry, till I come to thee, and shew thee what thou shalt do."],
    ['1Samuel','1Sa','10','009','9',"And it was \emph{so}, that when he had turned his back to go from Samuel, God gave him another heart: and all those signs came to pass that day."],
    ['1Samuel','1Sa','10','010','10',"And when they came thither to the hill, behold, a company of prophets met him; and the Spirit of God came upon him, and he prophesied among them."],
    ['1Samuel','1Sa','10','011','11',"And it came to pass, when all that knew him beforetime saw that, behold, he prophesied among the prophets, then the people said one to another, What \emph{is} this \emph{that} is come unto the son of Kish? \emph{Is} Saul also among the prophets?"],
    ['1Samuel','1Sa','10','012','12',"And one of the same place answered and said, But who \emph{is} their father? Therefore it became a proverb, \emph{Is} Saul also among the prophets?"],
    ['1Samuel','1Sa','10','013','13',"And when he had made an end of prophesying, he came to the high place."],
    ['1Samuel','1Sa','10','014','14',"And Saul's uncle said unto him and to his servant, Whither went ye? And he said, To seek the asses: and when we saw that \emph{they} \emph{were} no where, we came to Samuel."],
    ['1Samuel','1Sa','10','015','15',"And Saul's uncle said, Tell me, I pray thee, what Samuel said unto you."],
    ['1Samuel','1Sa','10','016','16',"And Saul said unto his uncle, He told us plainly that the asses were found. But of the matter of the kingdom, whereof Samuel spake, he told him not."],
    ['1Samuel','1Sa','10','017','17',"And Samuel called the people together unto the LORD to Mizpeh;"],
    ['1Samuel','1Sa','10','018','18',"And said unto the children of Israel, Thus saith the LORD God of Israel, I brought up Israel out of Egypt, and delivered you out of the hand of the Egyptians, and out of the hand of all kingdoms, \emph{and} of them that oppressed you:"],
    ['1Samuel','1Sa','10','019','19',"And ye have this day rejected your God, who himself saved you out of all your adversities and your tribulations; and ye have said unto him, \emph{Nay}, but set a king over us. Now therefore present yourselves before the LORD by your tribes, and by your thousands."],
    ['1Samuel','1Sa','10','020','20',"And when Samuel had caused all the tribes of Israel to come near, the tribe of Benjamin was taken."],
    ['1Samuel','1Sa','10','021','21',"When he had caused the tribe of Benjamin to come near by their families, the family of Matri was taken, and Saul the son of Kish was taken: and when they sought him, he could not be found."],
    ['1Samuel','1Sa','10','022','22',"Therefore they enquired of the LORD further, if the man should yet come thither. And the LORD answered, Behold, he hath hid himself among the stuff."],
    ['1Samuel','1Sa','10','023','23',"And they ran and fetched him thence: and when he stood among the people, he was higher than any of the people from his shoulders and upward."],
    ['1Samuel','1Sa','10','024','24',"And Samuel said to all the people, See ye him whom the LORD hath chosen, that \emph{there} \emph{is} none like him among all the people? And all the people shouted, and said, God save the king."],
    ['1Samuel','1Sa','10','025','25',"Then Samuel told the people the manner of the kingdom, and wrote \emph{it} in a book, and laid \emph{it} up before the LORD. And Samuel sent all the people away, every man to his house."],
    ['1Samuel','1Sa','10','026','26',"And Saul also went home to Gibeah; and there went with him a band of men, whose hearts God had touched."],
    ['1Samuel','1Sa','10','027','27',"But the children of Belial said, How shall this man save us? And they despised him, and brought him no presents. But he held his peace."]
]

FirstSamuel11 = [
    ['1Samuel','1Sa','11','001','1',"Then Nahash the Ammonite came up, and encamped against Jabesh-gilead: and all the men of Jabesh said unto Nahash, Make a covenant with us, and we will serve thee."],
    ['1Samuel','1Sa','11','002','2',"And Nahash the Ammonite answered them, On this \emph{condition} will I make \emph{a} \emph{covenant} with you, that I may thrust out all your right eyes, and lay it \emph{for} a reproach upon all Israel."],
    ['1Samuel','1Sa','11','003','3',"And the elders of Jabesh said unto him, Give us seven days' respite, that we may send messengers unto all the coasts of Israel: and then, if \emph{there} \emph{be} no man to save us, we will come out to thee."],
    ['1Samuel','1Sa','11','004','4',"Then came the messengers to Gibeah of Saul, and told the tidings in the ears of the people: and all the people lifted up their voices, and wept."],
    ['1Samuel','1Sa','11','005','5',"And, behold, Saul came after the herd out of the field; and Saul said, What \emph{aileth} the people that they weep? And they told him the tidings of the men of Jabesh."],
    ['1Samuel','1Sa','11','006','6',"And the Spirit of God came upon Saul when he heard those tidings, and his anger was kindled greatly."],
    ['1Samuel','1Sa','11','007','7',"And he took a yoke of oxen, and hewed them in pieces, and sent \emph{them} throughout all the coasts of Israel by the hands of messengers, saying, Whosoever cometh not forth after Saul and after Samuel, so shall it be done unto his oxen. And the fear of the LORD fell on the people, and they came out with one consent."],
    ['1Samuel','1Sa','11','008','8',"And when he numbered them in Bezek, the children of Israel were three hundred thousand, and the men of Judah thirty thousand."],
    ['1Samuel','1Sa','11','009','9',"And they said unto the messengers that came, Thus shall ye say unto the men of Jabesh-gilead, To morrow, by \emph{that} \emph{time} the sun be hot, ye shall have help. And the messengers came and shewed \emph{it} to the men of Jabesh; and they were glad."],
    ['1Samuel','1Sa','11','010','10',"Therefore the men of Jabesh said, To morrow we will come out unto you, and ye shall do with us all that seemeth good unto you."],
    ['1Samuel','1Sa','11','011','11',"And it was \emph{so} on the morrow, that Saul put the people in three companies; and they came into the midst of the host in the morning watch, and slew the Ammonites until the heat of the day: and it came to pass, that they which remained were scattered, so that two of them were not left together."],
    ['1Samuel','1Sa','11','012','12',"And the people said unto Samuel, Who \emph{is} he that said, Shall Saul reign over us? bring the men, that we may put them to death."],
    ['1Samuel','1Sa','11','013','13',"And Saul said, There shall not a man be put to death this day: for to day the LORD hath wrought salvation in Israel."],
    ['1Samuel','1Sa','11','014','14',"Then said Samuel to the people, Come, and let us go to Gilgal, and renew the kingdom there."],
    ['1Samuel','1Sa','11','015','15',"And all the people went to Gilgal; and there they made Saul king before the LORD in Gilgal; and there they sacrificed sacrifices of peace offerings before the LORD; and there Saul and all the men of Israel rejoiced greatly."]
]

FirstSamuel12 = [
    ['1Samuel','1Sa','12','001','1',"And Samuel said unto all Israel, Behold, I have hearkened unto your voice in all that ye said unto me, and have made a king over you."],
    ['1Samuel','1Sa','12','002','2',"And now, behold, the king walketh before you: and I am old and grayheaded; and, behold, my sons \emph{are} with you: and I have walked before you from my childhood unto this day."],
    ['1Samuel','1Sa','12','003','3',"Behold, here I \emph{am}: witness against me before the LORD, and before his anointed: whose ox have I taken? or whose ass have I taken? or whom have I defrauded? whom have I oppressed? or of whose hand have I received \emph{any} bribe to blind mine eyes therewith? and I will restore it you."],
    ['1Samuel','1Sa','12','004','4',"And they said, Thou hast not defrauded us, nor oppressed us, neither hast thou taken ought of any man's hand."],
    ['1Samuel','1Sa','12','005','5',"And he said unto them, The LORD \emph{is} witness against you, and his anointed \emph{is} witness this day, that ye have not found ought in my hand. And they answered, \emph{He} \emph{is} witness."],
    ['1Samuel','1Sa','12','006','6',"And Samuel said unto the people, \emph{It} \emph{is} the LORD that advanced Moses and Aaron, and that brought your fathers up out of the land of Egypt."],
    ['1Samuel','1Sa','12','007','7',"Now therefore stand still, that I may reason with you before the LORD of all the righteous acts of the LORD, which he did to you and to your fathers."],
    ['1Samuel','1Sa','12','008','8',"When Jacob was come into Egypt, and your fathers cried unto the LORD, then the LORD sent Moses and Aaron, which brought forth your fathers out of Egypt, and made them dwell in this place."],
    ['1Samuel','1Sa','12','009','9',"And when they forgat the LORD their God, he sold them into the hand of Sisera, captain of the host of Hazor, and into the hand of the Philistines, and into the hand of the king of Moab, and they fought against them."],
    ['1Samuel','1Sa','12','010','10',"And they cried unto the LORD, and said, We have sinned, because we have forsaken the LORD, and have served Baalim and Ashtaroth: but now deliver us out of the hand of our enemies, and we will serve thee."],
    ['1Samuel','1Sa','12','011','11',"And the LORD sent Jerubbaal, and Bedan, and Jephthah, and Samuel, and delivered you out of the hand of your enemies on every side, and ye dwelled safe."],
    ['1Samuel','1Sa','12','012','12',"And when ye saw that Nahash the king of the children of Ammon came against you, ye said unto me, Nay; but a king shall reign over us: when the LORD your God \emph{was} your king."],
    ['1Samuel','1Sa','12','013','13',"Now therefore behold the king whom ye have chosen, \emph{and} whom ye have desired! and, behold, the LORD hath set a king over you."],
    ['1Samuel','1Sa','12','014','14',"If ye will fear the LORD, and serve him, and obey his voice, and not rebel against the commandment of the LORD, then shall both ye and also the king that reigneth over you continue following the LORD your God:"],
    ['1Samuel','1Sa','12','015','15',"But if ye will not obey the voice of the LORD, but rebel against the commandment of the LORD, then shall the hand of the LORD be against you, as \emph{it} \emph{was} against your fathers."],
    ['1Samuel','1Sa','12','016','16',"Now therefore stand and see this great thing, which the LORD will do before your eyes."],
    ['1Samuel','1Sa','12','017','17',"\emph{Is} \emph{it} not wheat harvest to day? I will call unto the LORD, and he shall send thunder and rain; that ye may perceive and see that your wickedness \emph{is} great, which ye have done in the sight of the LORD, in asking you a king."],
    ['1Samuel','1Sa','12','018','18',"So Samuel called unto the LORD; and the LORD sent thunder and rain that day: and all the people greatly feared the LORD and Samuel."],
    ['1Samuel','1Sa','12','019','19',"And all the people said unto Samuel, Pray for thy servants unto the LORD thy God, that we die not: for we have added unto all our sins \emph{this} evil, to ask us a king."],
    ['1Samuel','1Sa','12','020','20',"And Samuel said unto the people, Fear not: ye have done all this wickedness: yet turn not aside from following the LORD, but serve the LORD with all your heart;"],
    ['1Samuel','1Sa','12','021','21',"And turn ye not aside: for \emph{then} \emph{should} \emph{ye} \emph{go} after vain \emph{things}, which cannot profit nor deliver; for they \emph{are} vain."],
    ['1Samuel','1Sa','12','022','22',"For the LORD will not forsake his people for his great name's sake: because it hath pleased the LORD to make you his people."],
    ['1Samuel','1Sa','12','023','23',"Moreover as for me, God forbid that I should sin against the LORD in ceasing to pray for you: but I will teach you the good and the right way:"],
    ['1Samuel','1Sa','12','024','24',"Only fear the LORD, and serve him in truth with all your heart: for consider how great \emph{things} he hath done for you."],
    ['1Samuel','1Sa','12','025','25',"But if ye shall still do wickedly, ye shall be consumed, both ye and your king."]
]

FirstSamuel13 = [
    ['1Samuel','1Sa','13','001','1',"Saul reigned one year; and when he had reigned two years over Israel,"],
['1Samuel','1Sa','13','002','2',"Saul chose him three thousand \emph{men} of Israel; \emph{whereof} two thousand were with Saul in Michmash and in mount Beth-el, and a thousand were with Jonathan in Gibeah of Benjamin: and the rest of the people he sent every man to his tent."],
['1Samuel','1Sa','13','003','3',"And Jonathan smote the garrison of the Philistines that \emph{was} in Geba, and the Philistines heard \emph{of} \emph{it}. And Saul blew the trumpet throughout all the land, saying, Let the Hebrews hear."],
['1Samuel','1Sa','13','004','4',"And all Israel heard say \emph{that} Saul had smitten a garrison of the Philistines, and \emph{that} Israel also was had in abomination with the Philistines. And the people were called together after Saul to Gilgal."],
['1Samuel','1Sa','13','005','5',"And the Philistines gathered themselves together to fight with Israel, thirty thousand chariots, and six thousand horsemen, and people as the sand which \emph{is} on the sea shore in multitude: and they came up, and pitched in Michmash, eastward from Beth-aven."],
['1Samuel','1Sa','13','006','6',"When the men of Israel saw that they were in a strait, (for the people were distressed,) then the people did hide themselves in caves, and in thickets, and in rocks, and in high places, and in pits."],
['1Samuel','1Sa','13','007','7',"And \emph{some} \emph{of} the Hebrews went over Jordan to the land of Gad and Gilead. As for Saul, he \emph{was} yet in Gilgal, and all the people followed him trembling."],
['1Samuel','1Sa','13','008','8',"And he tarried seven days, according to the set time that Samuel \emph{had} \emph{appointed}: but Samuel came not to Gilgal; and the people were scattered from him."],
['1Samuel','1Sa','13','009','9',"And Saul said, Bring hither a burnt offering to me, and peace offerings. And he offered the burnt offering."],
['1Samuel','1Sa','13','010','10',"And it came to pass, that as soon as he had made an end of offering the burnt offering, behold, Samuel came; and Saul went out to meet him, that he might salute him."],
['1Samuel','1Sa','13','011','11',"And Samuel said, What hast thou done? And Saul said, Because I saw that the people were scattered from me, and \emph{that} thou camest not within the days appointed, and \emph{that} the Philistines gathered themselves together at Michmash;"],
['1Samuel','1Sa','13','012','12',"Therefore said I, The Philistines will come down now upon me to Gilgal, and I have not made supplication unto the LORD: I forced myself therefore, and offered a burnt offering."],
['1Samuel','1Sa','13','013','13',"And Samuel said to Saul, Thou hast done foolishly: thou hast not kept the commandment of the LORD thy God, which he commanded thee: for now would the LORD have established thy kingdom upon Israel for ever."],
['1Samuel','1Sa','13','014','14',"But now thy kingdom shall not continue: the LORD hath sought him a man after his own heart, and the LORD hath commanded him \emph{to} \emph{be} captain over his people, because thou hast not kept \emph{that} which the LORD commanded thee."],
['1Samuel','1Sa','13','015','15',"And Samuel arose, and gat him up from Gilgal unto Gibeah of Benjamin. And Saul numbered the people \emph{that} \emph{were} present with him, about six hundred men."],
['1Samuel','1Sa','13','016','16',"And Saul, and Jonathan his son, and the people \emph{that} \emph{were} present with them, abode in Gibeah of Benjamin: but the Philistines encamped in Michmash."],
['1Samuel','1Sa','13','017','17',"And the spoilers came out of the camp of the Philistines in three companies: one company turned unto the way \emph{that} \emph{leadeth} \emph{to} Ophrah, unto the land of Shual:"],
['1Samuel','1Sa','13','018','18',"And another company turned the way \emph{to} Beth-horon: and another company turned \emph{to} the way of the border that looketh to the valley of Zeboim toward the wilderness."],
['1Samuel','1Sa','13','019','19',"Now there was no smith found throughout all the land of Israel: for the Philistines said, Lest the Hebrews make \emph{them} swords or spears:"],
['1Samuel','1Sa','13','020','20',"But all the Israelites went down to the Philistines, to sharpen every man his share, and his coulter, and his axe, and his mattock."],
['1Samuel','1Sa','13','021','21',"Yet they had a file for the mattocks, and for the coulters, and for the forks, and for the axes, and to sharpen the goads."],
['1Samuel','1Sa','13','022','22',"So it came to pass in the day of battle, that there was neither sword nor spear found in the hand of any of the people that \emph{were} with Saul and Jonathan: but with Saul and with Jonathan his son was there found."],
['1Samuel','1Sa','13','023','23',"And the garrison of the Philistines went out to the passage of Michmash."]
]

FirstSamuel14 = [
    ['1Samuel','1Sa','14','001','1',"Now it came to pass upon a day, that Jonathan the son of Saul said unto the young man that bare his armour, Come, and let us go over to the Philistines' garrison, that \emph{is} on the other side. But he told not his father."],
    ['1Samuel','1Sa','14','002','2',"And Saul tarried in the uttermost part of Gibeah under a pomegranate tree which \emph{is} in Migron: and the people that \emph{were} with him \emph{were} about six hundred men;"],
    ['1Samuel','1Sa','14','003','3',"And Ahiah, the son of Ahitub, I-chabod's brother, the son of Phinehas, the son of Eli, the LORD'S priest in Shiloh, wearing an ephod. And the people knew not that Jonathan was gone."],
    ['1Samuel','1Sa','14','004','4',"And between the passages, by which Jonathan sought to go over unto the Philistines' garrison, \emph{there} \emph{was} a sharp rock on the one side, and a sharp rock on the other side: and the name of the one \emph{was} Bozez, and the name of the other Seneh."],
    ['1Samuel','1Sa','14','005','5',"The forefront of the one \emph{was} situate northward over against Michmash, and the other southward over against Gibeah."],
    ['1Samuel','1Sa','14','006','6',"And Jonathan said to the young man that bare his armour, Come, and let us go over unto the garrison of these uncircumcised: it may be that the LORD will work for us: for \emph{there} \emph{is} no restraint to the LORD to save by many or by few."],
    ['1Samuel','1Sa','14','007','7',"And his armourbearer said unto him, Do all that \emph{is} in thine heart: turn thee; behold, I \emph{am} with thee according to thy heart."],
['1Samuel','1Sa','14','008','8',"Then said Jonathan, Behold, we will pass over unto \emph{these} men, and we will discover ourselves unto them."],
['1Samuel','1Sa','14','009','9',"If they say thus unto us, Tarry until we come to you; then we will stand still in our place, and will not go up unto them."],
['1Samuel','1Sa','14','010','10',"But if they say thus, Come up unto us; then we will go up: for the LORD hath delivered them into our hand: and this \emph{shall} \emph{be} a sign unto us."],
['1Samuel','1Sa','14','011','11',"And both of them discovered themselves unto the garrison of the Philistines: and the Philistines said, Behold, the Hebrews come forth out of the holes where they had hid themselves."],
['1Samuel','1Sa','14','012','12',"And the men of the garrison answered Jonathan and his armourbearer, and said, Come up to us, and we will shew you a thing. And Jonathan said unto his armourbearer, Come up after me: for the LORD hath delivered them into the hand of Israel."],
['1Samuel','1Sa','14','013','13',"And Jonathan climbed up upon his hands and upon his feet, and his armourbearer after him: and they fell before Jonathan; and his armourbearer slew after him."],
['1Samuel','1Sa','14','014','14',"And that first slaughter, which Jonathan and his armourbearer made, was about twenty men, within as it were an half acre of land, \emph{which} a yoke \emph{of} \emph{oxen} \emph{might} \emph{plow}."],
['1Samuel','1Sa','14','015','15',"And there was trembling in the host, in the field, and among all the people: the garrison, and the spoilers, they also trembled, and the earth quaked: so it was a very great trembling."],
['1Samuel','1Sa','14','016','16',"And the watchmen of Saul in Gibeah of Benjamin looked; and, behold, the multitude melted away, and they went on beating down \emph{one} \emph{another}."],
['1Samuel','1Sa','14','017','17',"Then said Saul unto the people that \emph{were} with him, Number now, and see who is gone from us. And when they had numbered, behold, Jonathan and his armourbearer \emph{were} not \emph{there}."],
['1Samuel','1Sa','14','018','18',"And Saul said unto Ahiah, Bring hither the ark of God. For the ark of God was at that time with the children of Israel."],
['1Samuel','1Sa','14','019','19',"And it came to pass, while Saul talked unto the priest, that the noise that \emph{was} in the host of the Philistines went on and increased: and Saul said unto the priest, Withdraw thine hand."],
['1Samuel','1Sa','14','020','20',"And Saul and all the people that \emph{were} with him assembled themselves, and they came to the battle: and, behold, every man's sword was against his fellow, \emph{and} \emph{there} \emph{was} a very great discomfiture."],
['1Samuel','1Sa','14','021','21',"Moreover the Hebrews \emph{that} were with the Philistines before that time, which went up with them into the camp \emph{from} \emph{the} \emph{country} round about, even they also \emph{turned} to be with the Israelites that \emph{were} with Saul and Jonathan."],
['1Samuel','1Sa','14','022','22',"Likewise all the men of Israel which had hid themselves in mount Ephraim, \emph{when} they heard that the Philistines fled, even they also followed hard after them in the battle."],
['1Samuel','1Sa','14','023','23',"So the LORD saved Israel that day: and the battle passed over unto Beth-aven."],
['1Samuel','1Sa','14','024','24',"And the men of Israel were distressed that day: for Saul had adjured the people, saying, Cursed \emph{be} the man that eateth \emph{any} food until evening, that I may be avenged on mine enemies. So none of the people tasted \emph{any} food."],
['1Samuel','1Sa','14','025','25',"And all \emph{they} \emph{of} the land came to a wood; and there was honey upon the ground."],
['1Samuel','1Sa','14','026','26',"And when the people were come into the wood, behold, the honey dropped; but no man put his hand to his mouth: for the people feared the oath."],
['1Samuel','1Sa','14','027','27',"But Jonathan heard not when his father charged the people with the oath: wherefore he put forth the end of the rod that \emph{was} in his hand, and dipped it in an honeycomb, and put his hand to his mouth; and his eyes were enlightened."],
['1Samuel','1Sa','14','028','28',"Then answered one of the people, and said, Thy father straitly charged the people with an oath, saying, Cursed \emph{be} the man that eateth \emph{any} food this day. And the people were faint."],
['1Samuel','1Sa','14','029','29',"Then said Jonathan, My father hath troubled the land: see, I pray you, how mine eyes have been enlightened, because I tasted a little of this honey."],
['1Samuel','1Sa','14','030','30',"How much more, if haply the people had eaten freely to day of the spoil of their enemies which they found? for had there not been now a much greater slaughter among the Philistines?"],
['1Samuel','1Sa','14','031','31',"And they smote the Philistines that day from Michmash to Aijalon: and the people were very faint."],
['1Samuel','1Sa','14','032','32',"And the people flew upon the spoil, and took sheep, and oxen, and calves, and slew \emph{them} on the ground: and the people did eat \emph{them} with the blood."],
['1Samuel','1Sa','14','033','33',"Then they told Saul, saying, Behold, the people sin against the LORD, in that they eat with the blood. And he said, Ye have transgressed: roll a great stone unto me this day."],
['1Samuel','1Sa','14','034','34',"And Saul said, Disperse yourselves among the people, and say unto them, Bring me hither every man his ox, and every man his sheep, and slay \emph{them} here, and eat; and sin not against the LORD in eating with the blood. And all the people brought every man his ox with him that night, and slew \emph{them} there."],
['1Samuel','1Sa','14','035','35',"And Saul built an altar unto the LORD: the same was the first altar that he built unto the LORD."],
['1Samuel','1Sa','14','036','36',"And Saul said, Let us go down after the Philistines by night, and spoil them until the morning light, and let us not leave a man of them. And they said, Do whatsoever seemeth good unto thee. Then said the priest, Let us draw near hither unto God."],
['1Samuel','1Sa','14','037','37',"And Saul asked counsel of God, Shall I go down after the Philistines? wilt thou deliver them into the hand of Israel? But he answered him not that day."],
['1Samuel','1Sa','14','038','38',"And Saul said, Draw ye near hither, all the chief of the people: and know and see wherein this sin hath been this day."],
['1Samuel','1Sa','14','039','39',"For, \emph{as} the LORD liveth, which saveth Israel, though it be in Jonathan my son, he shall surely die. But \emph{there} \emph{was} not a man among all the people \emph{that} answered him."],
['1Samuel','1Sa','14','040','40',"Then said he unto all Israel, Be ye on one side, and I and Jonathan my son will be on the other side. And the people said unto Saul, Do what seemeth good unto thee."],
['1Samuel','1Sa','14','041','41',"Therefore Saul said unto the LORD God of Israel, Give a perfect \emph{lot}. And Saul and Jonathan were taken: but the people escaped."],
['1Samuel','1Sa','14','042','42',"And Saul said, Cast \emph{lots} between me and Jonathan my son. And Jonathan was taken."],
['1Samuel','1Sa','14','043','43',"Then Saul said to Jonathan, Tell me what thou hast done. And Jonathan told him, and said, I did but taste a little honey with the end of the rod that \emph{was} in mine hand, \emph{and}, lo, I must die."],
['1Samuel','1Sa','14','044','44',"And Saul answered, God do so and more also: for thou shalt surely die, Jonathan."],
['1Samuel','1Sa','14','045','45',"And the people said unto Saul, Shall Jonathan die, who hath wrought this great salvation in Israel? God forbid: \emph{as} the LORD liveth, there shall not one hair of his head fall to the ground; for he hath wrought with God this day. So the people rescued Jonathan, that he died not."],
['1Samuel','1Sa','14','046','46',"Then Saul went up from following the Philistines: and the Philistines went to their own place."],
['1Samuel','1Sa','14','047','47',"So Saul took the kingdom over Israel, and fought against all his enemies on every side, against Moab, and against the children of Ammon, and against Edom, and against the kings of Zobah, and against the Philistines: and whithersoever he turned himself, he vexed \emph{them}."],
['1Samuel','1Sa','14','048','48',"And he gathered an host, and smote the Amalekites, and delivered Israel out of the hands of them that spoiled them."],
['1Samuel','1Sa','14','049','49',"Now the sons of Saul were Jonathan, and Ishui, and Melchi-shua: and the names of his two daughters \emph{were} \emph{these}; the name of the firstborn Merab, and the name of the younger Michal:"],
    ['1Samuel','1Sa','14','050','50',"And the name of Saul's wife \emph{was} Ahinoam, the daughter of Ahimaaz: and the name of the captain of his host \emph{was} Abner, the son of Ner, Saul's uncle."],
    ['1Samuel','1Sa','14','051','51',"And Kish \emph{was} the father of Saul; and Ner the father of Abner \emph{was} the son of Abiel."],
    ['1Samuel','1Sa','14','052','52',"And there was sore war against the Philistines all the days of Saul: and when Saul saw any strong man, or any valiant man, he took him unto him."]
]

FirstSamuel15 = [
    ['1Samuel','1Sa','15','001','1',"Samuel also said unto Saul, The LORD sent me to anoint thee \emph{to} \emph{be} king over his people, over Israel: now therefore hearken thou unto the voice of the words of the LORD."],
    ['1Samuel','1Sa','15','002','2',"Thus saith the LORD of hosts, I remember \emph{that} which Amalek did to Israel, how he laid \emph{wait} for him in the way, when he came up from Egypt."],
    ['1Samuel','1Sa','15','003','3',"Now go and smite Amalek, and utterly destroy all that they have, and spare them not; but slay both man and woman, infant and suckling, ox and sheep, camel and ass."],
    ['1Samuel','1Sa','15','004','4',"And Saul gathered the people together, and numbered them in Telaim, two hundred thousand footmen, and ten thousand men of Judah."],
    ['1Samuel','1Sa','15','005','5',"And Saul came to a city of Amalek, and laid wait in the valley."],
    ['1Samuel','1Sa','15','006','6',"And Saul said unto the Kenites, Go, depart, get you down from among the Amalekites, lest I destroy you with them: for ye shewed kindness to all the children of Israel, when they came up out of Egypt. So the Kenites departed from among the Amalekites."],
    ['1Samuel','1Sa','15','007','7',"And Saul smote the Amalekites from Havilah \emph{until} thou comest to Shur, that \emph{is} over against Egypt."],
    ['1Samuel','1Sa','15','008','8',"And he took Agag the king of the Amalekites alive, and utterly destroyed all the people with the edge of the sword."],
    ['1Samuel','1Sa','15','009','9',"But Saul and the people spared Agag, and the best of the sheep, and of the oxen, and of the fatlings, and the lambs, and all \emph{that} \emph{was} good, and would not utterly destroy them: but every thing \emph{that} \emph{was} vile and refuse, that they destroyed utterly."],
['1Samuel','1Sa','15','010','10',"Then came the word of the LORD unto Samuel, saying,"],
['1Samuel','1Sa','15','011','11',"It repenteth me that I have set up Saul \emph{to} \emph{be} king: for he is turned back from following me, and hath not performed my commandments. And it grieved Samuel; and he cried unto the LORD all night."],
['1Samuel','1Sa','15','012','12',"And when Samuel rose early to meet Saul in the morning, it was told Samuel, saying, Saul came to Carmel, and, behold, he set him up a place, and is gone about, and passed on, and gone down to Gilgal."],
['1Samuel','1Sa','15','013','13',"And Samuel came to Saul: and Saul said unto him, Blessed \emph{be} thou of the LORD: I have performed the commandment of the LORD."],
['1Samuel','1Sa','15','014','14',"And Samuel said, What \emph{meaneth} then this bleating of the sheep in mine ears, and the lowing of the oxen which I hear?"],
['1Samuel','1Sa','15','015','15',"And Saul said, They have brought them from the Amalekites: for the people spared the best of the sheep and of the oxen, to sacrifice unto the LORD thy God; and the rest we have utterly destroyed."],
['1Samuel','1Sa','15','016','16',"Then Samuel said unto Saul, Stay, and I will tell thee what the LORD hath said to me this night. And he said unto him, Say on."],
['1Samuel','1Sa','15','017','17',"And Samuel said, When thou \emph{wast} little in thine own sight, \emph{wast} thou not \emph{made} the head of the tribes of Israel, and the LORD anointed thee king over Israel?"],
['1Samuel','1Sa','15','018','18',"And the LORD sent thee on a journey, and said, Go and utterly destroy the sinners the Amalekites, and fight against them until they be consumed."],
['1Samuel','1Sa','15','019','19',"Wherefore then didst thou not obey the voice of the LORD, but didst fly upon the spoil, and didst evil in the sight of the LORD?"],
['1Samuel','1Sa','15','020','20',"And Saul said unto Samuel, Yea, I have obeyed the voice of the LORD, and have gone the way which the LORD sent me, and have brought Agag the king of Amalek, and have utterly destroyed the Amalekites."],
['1Samuel','1Sa','15','021','21',"But the people took of the spoil, sheep and oxen, the chief of the things which should have been utterly destroyed, to sacrifice unto the LORD thy God in Gilgal."],
['1Samuel','1Sa','15','022','22',"And Samuel said, Hath the LORD \emph{as} \emph{great} delight in burnt offerings and sacrifices, as in obeying the voice of the LORD? Behold, to obey \emph{is} better than sacrifice, \emph{and} to hearken than the fat of rams."],
['1Samuel','1Sa','15','023','23',"For rebellion \emph{is} \emph{as} the sin of witchcraft, and stubbornness \emph{is} \emph{as} iniquity and idolatry. Because thou hast rejected the word of the LORD, he hath also rejected thee from \emph{being} king."],
['1Samuel','1Sa','15','024','24',"And Saul said unto Samuel, I have sinned: for I have transgressed the commandment of the LORD, and thy words: because I feared the people, and obeyed their voice."],
['1Samuel','1Sa','15','025','25',"Now therefore, I pray thee, pardon my sin, and turn again with me, that I may worship the LORD."],
    ['1Samuel','1Sa','15','026','26',"And Samuel said unto Saul, I will not return with thee: for thou hast rejected the word of the LORD, and the LORD hath rejected thee from being king over Israel."],
    ['1Samuel','1Sa','15','027','27',"And as Samuel turned about to go away, he laid hold upon the skirt of his mantle, and it rent."],
    ['1Samuel','1Sa','15','028','28',"And Samuel said unto him, The LORD hath rent the kingdom of Israel from thee this day, and hath given it to a neighbour of thine, \emph{that} \emph{is} better than thou."],
    ['1Samuel','1Sa','15','029','29',"And also the Strength of Israel will not lie nor repent: for he \emph{is} not a man, that he should repent."],
    ['1Samuel','1Sa','15','030','30',"Then he said, I have sinned: \emph{yet} honour me now, I pray thee, before the elders of my people, and before Israel, and turn again with me, that I may worship the LORD thy God."],
    ['1Samuel','1Sa','15','031','31',"So Samuel turned again after Saul; and Saul worshipped the LORD."],
    ['1Samuel','1Sa','15','032','32',"Then said Samuel, Bring ye hither to me Agag the king of the Amalekites. And Agag came unto him delicately. And Agag said, Surely the bitterness of death is past."],
    ['1Samuel','1Sa','15','033','33',"And Samuel said, As thy sword hath made women childless, so shall thy mother be childless among women. And Samuel hewed Agag in pieces before the LORD in Gilgal."],
    ['1Samuel','1Sa','15','034','34',"Then Samuel went to Ramah; and Saul went up to his house to Gibeah of Saul."],
    ['1Samuel','1Sa','15','035','35',"And Samuel came no more to see Saul until the day of his death: nevertheless Samuel mourned for Saul: and the LORD repented that he had made Saul king over Israel."]
]

FirstSamuel16 = [
    ['1Samuel','1Sa','16','001','1',"And the LORD said unto Samuel, How long wilt thou mourn for Saul, seeing I have rejected him from reigning over Israel? fill thine horn with oil, and go, I will send thee to Jesse the Beth-lehemite: for I have provided me a king among his sons."],
    ['1Samuel','1Sa','16','002','2',"And Samuel said, How can I go? if Saul hear \emph{it}, he will kill me. And the LORD said, Take an heifer with thee, and say, I am come to sacrifice to the LORD."],
    ['1Samuel','1Sa','16','003','3',"And call Jesse to the sacrifice, and I will shew thee what thou shalt do: and thou shalt anoint unto me \emph{him} whom I name unto thee."],
    ['1Samuel','1Sa','16','004','4',"And Samuel did that which the LORD spake, and came to Beth-lehem. And the elders of the town trembled at his coming, and said, Comest thou peaceably?"],
    ['1Samuel','1Sa','16','005','5',"And he said, Peaceably: I am come to sacrifice unto the LORD: sanctify yourselves, and come with me to the sacrifice. And he sanctified Jesse and his sons, and called them to the sacrifice."],
    ['1Samuel','1Sa','16','006','6',"And it came to pass, when they were come, that he looked on Eliab, and said, Surely the LORD'S anointed \emph{is} before him."],
    ['1Samuel','1Sa','16','007','7',"But the LORD said unto Samuel, Look not on his countenance, or on the height of his stature; because I have refused him: for \emph{the} \emph{LORD} \emph{seeth} not as man seeth; for man looketh on the outward appearance, but the LORD looketh on the heart."],
    ['1Samuel','1Sa','16','008','8',"Then Jesse called Abinadab, and made him pass before Samuel. And he said, Neither hath the LORD chosen this."],
    ['1Samuel','1Sa','16','009','9',"Then Jesse made Shammah to pass by. And he said, Neither hath the LORD chosen this."],
    ['1Samuel','1Sa','16','010','10',"Again, Jesse made seven of his sons to pass before Samuel. And Samuel said unto Jesse, The LORD hath not chosen these."],
    ['1Samuel','1Sa','16','011','11',"And Samuel said unto Jesse, Are here all \emph{thy} children? And he said, There remaineth yet the youngest, and, behold, he keepeth the sheep. And Samuel said unto Jesse, Send and fetch him: for we will not sit down till he come hither."],
    ['1Samuel','1Sa','16','012','12',"And he sent, and brought him in. Now he \emph{was} ruddy, \emph{and} withal of a beautiful countenance, and goodly to look to. And the LORD said, Arise, anoint him: for this \emph{is} he."],
    ['1Samuel','1Sa','16','013','13',"Then Samuel took the horn of oil, and anointed him in the midst of his brethren: and the Spirit of the LORD came upon David from that day forward. So Samuel rose up, and went to Ramah."],
    ['1Samuel','1Sa','16','014','14',"But the Spirit of the LORD departed from Saul, and an evil spirit from the LORD troubled him."],
    ['1Samuel','1Sa','16','015','15',"And Saul's servants said unto him, Behold now, an evil spirit from God troubleth thee."],
    ['1Samuel','1Sa','16','016','16',"Let our lord now command thy servants, \emph{which} \emph{are} before thee, to seek out a man, \emph{who} \emph{is} a cunning player on an harp: and it shall come to pass, when the evil spirit from God is upon thee, that he shall play with his hand, and thou shalt be well."],
    ['1Samuel','1Sa','16','017','17',"And Saul said unto his servants, Provide me now a man that can play well, and bring \emph{him} to me."],
    ['1Samuel','1Sa','16','018','18',"Then answered one of the servants, and said, Behold, I have seen a son of Jesse the Beth-lehemite, \emph{that} \emph{is} cunning in playing, and a mighty valiant man, and a man of war, and prudent in matters, and a comely person, and the LORD \emph{is} with him."],
    ['1Samuel','1Sa','16','019','19',"Wherefore Saul sent messengers unto Jesse, and said, Send me David thy son, which \emph{is} with the sheep."],
    ['1Samuel','1Sa','16','020','20',"And Jesse took an ass \emph{laden} with bread, and a bottle of wine, and a kid, and sent \emph{them} by David his son unto Saul."],
    ['1Samuel','1Sa','16','021','21',"And David came to Saul, and stood before him: and he loved him greatly; and he became his armourbearer."],
    ['1Samuel','1Sa','16','022','22',"And Saul sent to Jesse, saying, Let David, I pray thee, stand before me; for he hath found favour in my sight."],
    ['1Samuel','1Sa','16','023','23',"And it came to pass, when the \emph{evil} spirit from God was upon Saul, that David took an harp, and played with his hand: so Saul was refreshed, and was well, and the evil spirit departed from him."]
]

FirstSamuel17 = [
    ['1Samuel','1Sa','17','001','1',"Now the Philistines gathered together their armies to battle, and were gathered together at Shochoh, which \emph{belongeth} to Judah, and pitched between Shochoh and Azekah, in Ephes-dammim."],
    ['1Samuel','1Sa','17','002','2',"And Saul and the men of Israel were gathered together, and pitched by the valley of Elah, and set the battle in array against the Philistines."],
    ['1Samuel','1Sa','17','003','3',"And the Philistines stood on a mountain on the one side, and Israel stood on a mountain on the other side: and \emph{there} \emph{was} a valley between them."],
    ['1Samuel','1Sa','17','004','4',"And there went out a champion out of the camp of the Philistines, named Goliath, of Gath, whose height \emph{was} six cubits and a span."],
    ['1Samuel','1Sa','17','005','5',"And \emph{he} \emph{had} an helmet of brass upon his head, and he \emph{was} armed with a coat of mail; and the weight of the coat \emph{was} five thousand shekels of brass."],
    ['1Samuel','1Sa','17','006','6',"And \emph{he} \emph{had} greaves of brass upon his legs, and a target of brass between his shoulders."],
    ['1Samuel','1Sa','17','007','7',"And the staff of his spear \emph{was} like a weaver's beam; and his spear's head \emph{weighed} six hundred shekels of iron: and one bearing a shield went before him."],
    ['1Samuel','1Sa','17','008','8',"And he stood and cried unto the armies of Israel, and said unto them, Why are ye come out to set \emph{your} battle in array? \emph{am} not I a Philistine, and ye servants to Saul? choose you a man for you, and let him come down to me."],
    ['1Samuel','1Sa','17','009','9',"If he be able to fight with me, and to kill me, then will we be your servants: but if I prevail against him, and kill him, then shall ye be our servants, and serve us."],
    ['1Samuel','1Sa','17','010','10',"And the Philistine said, I defy the armies of Israel this day; give me a man, that we may fight together."],
    ['1Samuel','1Sa','17','011','11',"When Saul and all Israel heard those words of the Philistine, they were dismayed, and greatly afraid."],
    ['1Samuel','1Sa','17','012','12',"Now David \emph{was} the son of that Ephrathite of Beth-lehem-judah, whose name \emph{was} Jesse; and he had eight sons: and the man went among men \emph{for} an old man in the days of Saul."],
    ['1Samuel','1Sa','17','013','13',"And the three eldest sons of Jesse went \emph{and} followed Saul to the battle: and the names of his three sons that went to the battle \emph{were} Eliab the firstborn, and next unto him Abinadab, and the third Shammah."],
    ['1Samuel','1Sa','17','014','14',"And David \emph{was} the youngest: and the three eldest followed Saul."],
    ['1Samuel','1Sa','17','015','15',"But David went and returned from Saul to feed his father's sheep at Beth-lehem."],
    ['1Samuel','1Sa','17','016','16',"And the Philistine drew near morning and evening, and presented himself forty days."],
    ['1Samuel','1Sa','17','017','17',"And Jesse said unto David his son, Take now for thy brethren an ephah of this parched \emph{corn}, and these ten loaves, and run to the camp to thy brethren;"],
    ['1Samuel','1Sa','17','018','18',"And carry these ten cheeses unto the captain of \emph{their} thousand, and look how thy brethren fare, and take their pledge."],
    ['1Samuel','1Sa','17','019','19',"Now Saul, and they, and all the men of Israel, \emph{were} in the valley of Elah, fighting with the Philistines."],
    ['1Samuel','1Sa','17','020','20',"And David rose up early in the morning, and left the sheep with a keeper, and took, and went, as Jesse had commanded him; and he came to the trench, as the host was going forth to the fight, and shouted for the battle."],
    ['1Samuel','1Sa','17','021','21',"For Israel and the Philistines had put the battle in array, army against army."],
    ['1Samuel','1Sa','17','022','22',"And David left his carriage in the hand of the keeper of the carriage, and ran into the army, and came and saluted his brethren."],
    ['1Samuel','1Sa','17','023','23',"And as he talked with them, behold, there came up the champion, the Philistine of Gath, Goliath by name, out of the armies of the Philistines, and spake according to the same words: and David heard \emph{them}."],
    ['1Samuel','1Sa','17','024','24',"And all the men of Israel, when they saw the man, fled from him, and were sore afraid."],
    ['1Samuel','1Sa','17','025','25',"And the men of Israel said, Have ye seen this man that is come up? surely to defy Israel is he come up: and it shall be, \emph{that} the man who killeth him, the king will enrich him with great riches, and will give him his daughter, and make his father's house free in Israel."],
    ['1Samuel','1Sa','17','026','26',"And David spake to the men that stood by him, saying, What shall be done to the man that killeth this Philistine, and taketh away the reproach from Israel? for who \emph{is} this uncircumcised Philistine, that he should defy the armies of the living God?"],
    ['1Samuel','1Sa','17','027','27',"And the people answered him after this manner, saying, So shall it be done to the man that killeth him."],
    ['1Samuel','1Sa','17','028','28',"And Eliab his eldest brother heard when he spake unto the men; and Eliab's anger was kindled against David, and he said, Why camest thou down hither? and with whom hast thou left those few sheep in the wilderness? I know thy pride, and the naughtiness of thine heart; for thou art come down that thou mightest see the battle."],
    ['1Samuel','1Sa','17','029','29',"And David said, What have I now done? \emph{Is} \emph{there} not a cause?"],
    ['1Samuel','1Sa','17','030','30',"And he turned from him toward another, and spake after the same manner: and the people answered him again after the former manner."],
    ['1Samuel','1Sa','17','031','31',"And when the words were heard which David spake, they rehearsed \emph{them} before Saul: and he sent for him."],
    ['1Samuel','1Sa','17','032','32',"And David said to Saul, Let no man's heart fail because of him; thy servant will go and fight with this Philistine."],
    ['1Samuel','1Sa','17','033','33',"And Saul said to David, Thou art not able to go against this Philistine to fight with him: for thou \emph{art} \emph{but} a youth, and he a man of war from his youth."],
    ['1Samuel','1Sa','17','034','34',"And David said unto Saul, Thy servant kept his father's sheep, and there came a lion, and a bear, and took a lamb out of the flock:"],
    ['1Samuel','1Sa','17','035','35',"And I went out after him, and smote him, and delivered \emph{it} out of his mouth: and when he arose against me, I caught \emph{him} by his beard, and smote him, and slew him."],
    ['1Samuel','1Sa','17','036','36',"Thy servant slew both the lion and the bear: and this uncircumcised Philistine shall be as one of them, seeing he hath defied the armies of the living God."],
    ['1Samuel','1Sa','17','037','37',"David said moreover, The LORD that delivered me out of the paw of the lion, and out of the paw of the bear, he will deliver me out of the hand of this Philistine. And Saul said unto David, Go, and the LORD be with thee."],
    ['1Samuel','1Sa','17','038','38',"And Saul armed David with his armour, and he put an helmet of brass upon his head; also he armed him with a coat of mail."],
    ['1Samuel','1Sa','17','039','39',"And David girded his sword upon his armour, and he assayed to go; for he had not proved \emph{it}. And David said unto Saul, I cannot go with these; for I have not proved \emph{them}. And David put them off him."],
    ['1Samuel','1Sa','17','040','40',"And he took his staff in his hand, and chose him five smooth stones out of the brook, and put them in a shepherd's bag which he had, even in a scrip; and his sling \emph{was} in his hand: and he drew near to the Philistine."],
    ['1Samuel','1Sa','17','041','41',"And the Philistine came on and drew near unto David; and the man that bare the shield \emph{went} before him."],
    ['1Samuel','1Sa','17','042','42',"And when the Philistine looked about, and saw David, he disdained him: for he was \emph{but} a youth, and ruddy, and of a fair countenance."],
    ['1Samuel','1Sa','17','043','43',"And the Philistine said unto David, \emph{Am} I a dog, that thou comest to me with staves? And the Philistine cursed David by his gods."],
    ['1Samuel','1Sa','17','044','44',"And the Philistine said to David, Come to me, and I will give thy flesh unto the fowls of the air, and to the beasts of the field."],
    ['1Samuel','1Sa','17','045','45',"Then said David to the Philistine, Thou comest to me with a sword, and with a spear, and with a shield: but I come to thee in the name of the LORD of hosts, the God of the armies of Israel, whom thou hast defied."],
    ['1Samuel','1Sa','17','046','46',"This day will the LORD deliver thee into mine hand; and I will smite thee, and take thine head from thee; and I will give the carcases of the host of the Philistines this day unto the fowls of the air, and to the wild beasts of the earth; that all the earth may know that there is a God in Israel."],
    ['1Samuel','1Sa','17','047','47',"And all this assembly shall know that the LORD saveth not with sword and spear: for the battle \emph{is} the LORD'S, and he will give you into our hands."],
    ['1Samuel','1Sa','17','048','48',"And it came to pass, when the Philistine arose, and came and drew nigh to meet David, that David hasted, and ran toward the army to meet the Philistine."],
    ['1Samuel','1Sa','17','049','49',"And David put his hand in his bag, and took thence a stone, and slang \emph{it}, and smote the Philistine in his forehead, that the stone sunk into his forehead; and he fell upon his face to the earth."],
    ['1Samuel','1Sa','17','050','50',"So David prevailed over the Philistine with a sling and with a stone, and smote the Philistine, and slew him; but \emph{there} \emph{was} no sword in the hand of David."],
    ['1Samuel','1Sa','17','051','51',"Therefore David ran, and stood upon the Philistine, and took his sword, and drew it out of the sheath thereof, and slew him, and cut off his head therewith. And when the Philistines saw their champion was dead, they fled."],
    ['1Samuel','1Sa','17','052','52',"And the men of Israel and of Judah arose, and shouted, and pursued the Philistines, until thou come to the valley, and to the gates of Ekron. And the wounded of the Philistines fell down by the way to Shaaraim, even unto Gath, and unto Ekron."],
    ['1Samuel','1Sa','17','053','53',"And the children of Israel returned from chasing after the Philistines, and they spoiled their tents."],
    ['1Samuel','1Sa','17','054','54',"And David took the head of the Philistine, and brought it to Jerusalem; but he put his armour in his tent."],
    ['1Samuel','1Sa','17','055','55',"And when Saul saw David go forth against the Philistine, he said unto Abner, the captain of the host, Abner, whose son \emph{is} this youth? And Abner said, \emph{As} thy soul liveth, O king, I cannot tell."],
    ['1Samuel','1Sa','17','056','56',"And the king said, Enquire thou whose son the stripling \emph{is}."],
    ['1Samuel','1Sa','17','057','57',"And as David returned from the slaughter of the Philistine, Abner took him, and brought him before Saul with the head of the Philistine in his hand."],
    ['1Samuel','1Sa','17','058','58',"And Saul said to him, Whose son \emph{art} thou, \emph{thou} young man? And David answered, I \emph{am} the son of thy servant Jesse the Beth-lehemite."]
]

FirstSamuel18 = [
    ['1Samuel','1Sa','18','001','1',"And it came to pass, when he had made an end of speaking unto Saul, that the soul of Jonathan was knit with the soul of David, and Jonathan loved him as his own soul."],
    ['1Samuel','1Sa','18','002','2',"And Saul took him that day, and would let him go no more home to his father's house."],
    ['1Samuel','1Sa','18','003','3',"Then Jonathan and David made a covenant, because he loved him as his own soul."],
    ['1Samuel','1Sa','18','004','4',"And Jonathan stripped himself of the robe that \emph{was} upon him, and gave it to David, and his garments, even to his sword, and to his bow, and to his girdle."],
    ['1Samuel','1Sa','18','005','5',"And David went out whithersoever Saul sent him, \emph{and} behaved himself wisely: and Saul set him over the men of war, and he was accepted in the sight of all the people, and also in the sight of Saul's servants."],
    ['1Samuel','1Sa','18','006','6',"And it came to pass as they came, when David was returned from the slaughter of the Philistine, that the women came out of all cities of Israel, singing and dancing, to meet king Saul, with tabrets, with joy, and with instruments of musick."],
    ['1Samuel','1Sa','18','007','7',"And the women answered \emph{one} \emph{another} as they played, and said, Saul hath slain his thousands, and David his ten thousands."],
    ['1Samuel','1Sa','18','008','8',"And Saul was very wroth, and the saying displeased him; and he said, They have ascribed unto David ten thousands, and to me they have ascribed \emph{but} thousands: and \emph{what} can he have more but the kingdom?"],
    ['1Samuel','1Sa','18','009','9',"And Saul eyed David from that day and forward."],
    ['1Samuel','1Sa','18','010','10',"And it came to pass on the morrow, that the evil spirit from God came upon Saul, and he prophesied in the midst of the house: and David played with his hand, as at other times: and \emph{there} \emph{was} a javelin in Saul's hand."],
    ['1Samuel','1Sa','18','011','11',"And Saul cast the javelin; for he said, I will smite David even to the wall \emph{with} \emph{it}. And David avoided out of his presence twice."],
    ['1Samuel','1Sa','18','012','12',"And Saul was afraid of David, because the LORD was with him, and was departed from Saul."],
    ['1Samuel','1Sa','18','013','13',"Therefore Saul removed him from him, and made him his captain over a thousand; and he went out and came in before the people."],
    ['1Samuel','1Sa','18','014','14',"And David behaved himself wisely in all his ways; and the LORD \emph{was} with him."],
    ['1Samuel','1Sa','18','015','15',"Wherefore when Saul saw that he behaved himself very wisely, he was afraid of him."],
    ['1Samuel','1Sa','18','016','16',"But all Israel and Judah loved David, because he went out and came in before them."],
    ['1Samuel','1Sa','18','017','17',"And Saul said to David, Behold my elder daughter Merab, her will I give thee to wife: only be thou valiant for me, and fight the LORD'S battles. For Saul said, Let not mine hand be upon him, but let the hand of the Philistines be upon him."],
    ['1Samuel','1Sa','18','018','18',"And David said unto Saul, Who \emph{am} I? and what \emph{is} my life, \emph{or} my father's family in Israel, that I should be son in law to the king?"],
    ['1Samuel','1Sa','18','019','19',"But it came to pass at the time when Merab Saul's daughter should have been given to David, that she was given unto Adriel the Meholathite to wife."],
    ['1Samuel','1Sa','18','020','20',"And Michal Saul's daughter loved David: and they told Saul, and the thing pleased him."],
    ['1Samuel','1Sa','18','021','21',"And Saul said, I will give him her, that she may be a snare to him, and that the hand of the Philistines may be against him. Wherefore Saul said to David, Thou shalt this day be my son in law in \emph{the} \emph{one} \emph{of} the twain."],
    ['1Samuel','1Sa','18','022','22',"And Saul commanded his servants, \emph{saying}, Commune with David secretly, and say, Behold, the king hath delight in thee, and all his servants love thee: now therefore be the king's son in law."],
    ['1Samuel','1Sa','18','023','23',"And Saul's servants spake those words in the ears of David. And David said, Seemeth it to you \emph{a} light \emph{thing} to be a king's son in law, seeing that I \emph{am} a poor man, and lightly esteemed?"],
    ['1Samuel','1Sa','18','024','24',"And the servants of Saul told him, saying, On this manner spake David."],
    ['1Samuel','1Sa','18','025','25',"And Saul said, Thus shall ye say to David, The king desireth not any dowry, but an hundred foreskins of the Philistines, to be avenged of the king's enemies. But Saul thought to make David fall by the hand of the Philistines."],
    ['1Samuel','1Sa','18','026','26',"And when his servants told David these words, it pleased David well to be the king's son in law: and the days were not expired."],
    ['1Samuel','1Sa','18','027','27',"Wherefore David arose and went, he and his men, and slew of the Philistines two hundred men; and David brought their foreskins, and they gave them in full tale to the king, that he might be the king's son in law. And Saul gave him Michal his daughter to wife."],
    ['1Samuel','1Sa','18','028','28',"And Saul saw and knew that the LORD \emph{was} with David, and \emph{that} Michal Saul's daughter loved him."],
    ['1Samuel','1Sa','18','029','29',"And Saul was yet the more afraid of David; and Saul became David's enemy continually."],
    ['1Samuel','1Sa','18','030','30',"Then the princes of the Philistines went forth: and it came to pass, after they went forth, \emph{that} David behaved himself more wisely than all the servants of Saul; so that his name was much set by."]
]

FirstSamuel19 = [
    ['1Samuel','1Sa','19','001','1',"And Saul spake to Jonathan his son, and to all his servants, that they should kill David."],
    ['1Samuel','1Sa','19','002','2',"But Jonathan Saul's son delighted much in David: and Jonathan told David, saying, Saul my father seeketh to kill thee: now therefore, I pray thee, take heed to thyself until the morning, and abide in a secret \emph{place}, and hide thyself:"],
    ['1Samuel','1Sa','19','003','3',"And I will go out and stand beside my father in the field where thou \emph{art}, and I will commune with my father of thee; and what I see, that I will tell thee."],
    ['1Samuel','1Sa','19','004','4',"And Jonathan spake good of David unto Saul his father, and said unto him, Let not the king sin against his servant, against David; because he hath not sinned against thee, and because his works \emph{have} \emph{been} to thee-ward very good:"],
    ['1Samuel','1Sa','19','005','5',"For he did put his life in his hand, and slew the Philistine, and the LORD wrought a great salvation for all Israel: thou sawest \emph{it}, and didst rejoice: wherefore then wilt thou sin against innocent blood, to slay David without a cause?"],
    ['1Samuel','1Sa','19','006','6',"And Saul hearkened unto the voice of Jonathan: and Saul sware, \emph{As} the LORD liveth, he shall not be slain."],
    ['1Samuel','1Sa','19','007','7',"And Jonathan called David, and Jonathan shewed him all those things. And Jonathan brought David to Saul, and he was in his presence, as in times past."],
    ['1Samuel','1Sa','19','008','8',"And there was war again: and David went out, and fought with the Philistines, and slew them with a great slaughter; and they fled from him."],
    ['1Samuel','1Sa','19','009','9',"And the evil spirit from the LORD was upon Saul, as he sat in his house with his javelin in his hand: and David played with \emph{his} hand."],
    ['1Samuel','1Sa','19','010','10',"And Saul sought to smite David even to the wall with the javelin; but he slipped away out of Saul's presence, and he smote the javelin into the wall: and David fled, and escaped that night."],
    ['1Samuel','1Sa','19','011','11',"Saul also sent messengers unto David's house, to watch him, and to slay him in the morning: and Michal David's wife told him, saying, If thou save not thy life to night, to morrow thou shalt be slain."],
    ['1Samuel','1Sa','19','012','12',"So Michal let David down through a window: and he went, and fled, and escaped."],
    ['1Samuel','1Sa','19','013','13',"And Michal took an image, and laid \emph{it} in the bed, and put a pillow of goats' \emph{hair} for his bolster, and covered \emph{it} with a cloth."],
    ['1Samuel','1Sa','19','014','14',"And when Saul sent messengers to take David, she said, He \emph{is} sick."],
    ['1Samuel','1Sa','19','015','15',"And Saul sent the messengers \emph{again} to see David, saying, Bring him up to me in the bed, that I may slay him."],
    ['1Samuel','1Sa','19','016','16',"And when the messengers were come in, behold, \emph{there} \emph{was} an image in the bed, with a pillow of goats' \emph{hair} for his bolster."],
    ['1Samuel','1Sa','19','017','17',"And Saul said unto Michal, Why hast thou deceived me so, and sent away mine enemy, that he is escaped? And Michal answered Saul, He said unto me, Let me go; why should I kill thee?"],
    ['1Samuel','1Sa','19','018','18',"So David fled, and escaped, and came to Samuel to Ramah, and told him all that Saul had done to him. And he and Samuel went and dwelt in Naioth."],
    ['1Samuel','1Sa','19','019','19',"And it was told Saul, saying, Behold, David \emph{is} at Naioth in Ramah."],
    ['1Samuel','1Sa','19','020','20',"And Saul sent messengers to take David: and when they saw the company of the prophets prophesying, and Samuel standing \emph{as} appointed over them, the Spirit of God was upon the messengers of Saul, and they also prophesied."],
    ['1Samuel','1Sa','19','021','21',"And when it was told Saul, he sent other messengers, and they prophesied likewise. And Saul sent messengers again the third time, and they prophesied also."],
    ['1Samuel','1Sa','19','022','22',"Then went he also to Ramah, and came to a great well that \emph{is} in Sechu: and he asked and said, Where \emph{are} Samuel and David? And \emph{one} said, Behold, \emph{they} \emph{be} at Naioth in Ramah."],
    ['1Samuel','1Sa','19','023','23',"And he went thither to Naioth in Ramah: and the Spirit of God was upon him also, and he went on, and prophesied, until he came to Naioth in Ramah."],
    ['1Samuel','1Sa','19','024','24',"And he stripped off his clothes also, and prophesied before Samuel in like manner, and lay down naked all that day and all that night. Wherefore they say, \emph{Is} Saul also among the prophets?"]
]

FirstSamuel20 = [
    ['1Samuel','1Sa','20','001','1',"And David fled from Naioth in Ramah, and came and said before Jonathan, What have I done? what \emph{is} mine iniquity? and what \emph{is} my sin before thy father, that he seeketh my life?"],
    ['1Samuel','1Sa','20','002','2',"And he said unto him, God forbid; thou shalt not die: behold, my father will do nothing either great or small, but that he will shew it me: and why should my father hide this thing from me? it \emph{is} not \emph{so}."],
    ['1Samuel','1Sa','20','003','3',"And David sware moreover, and said, Thy father certainly knoweth that I have found grace in thine eyes; and he saith, Let not Jonathan know this, lest he be grieved: but truly \emph{as} the LORD liveth, and \emph{as} thy soul liveth, \emph{there} \emph{is} but a step between me and death."],
    ['1Samuel','1Sa','20','004','4',"Then said Jonathan unto David, Whatsoever thy soul desireth, I will even do \emph{it} for thee."],
    ['1Samuel','1Sa','20','005','5',"And David said unto Jonathan, Behold, to morrow \emph{is} the new moon, and I should not fail to sit with the king at meat: but let me go, that I may hide myself in the field unto the third \emph{day} at even."],
    ['1Samuel','1Sa','20','006','6',"If thy father at all miss me, then say, David earnestly asked \emph{leave} of me that he might run to Beth-lehem his city: for \emph{there} \emph{is} a yearly sacrifice there for all the family."],
    ['1Samuel','1Sa','20','007','7',"If he say thus, \emph{It} \emph{is} well; thy servant shall have peace: but if he be very wroth, \emph{then} be sure that evil is determined by him."],
    ['1Samuel','1Sa','20','008','8',"Therefore thou shalt deal kindly with thy servant; for thou hast brought thy servant into a covenant of the LORD with thee: notwithstanding, if there be in me iniquity, slay me thyself; for why shouldest thou bring me to thy father?"],
    ['1Samuel','1Sa','20','009','9',"And Jonathan said, Far be it from thee: for if I knew certainly that evil were determined by my father to come upon thee, then would not I tell it thee?"],
    ['1Samuel','1Sa','20','010','10',"Then said David to Jonathan, Who shall tell me? or what \emph{if} thy father answer thee roughly?"],
    ['1Samuel','1Sa','20','011','11',"And Jonathan said unto David, Come, and let us go out into the field. And they went out both of them into the field."],
    ['1Samuel','1Sa','20','012','12',"And Jonathan said unto David, O LORD God of Israel, when I have sounded my father about to morrow any time, \emph{or} the third \emph{day}, and, behold, \emph{if} \emph{there} \emph{be} good toward David, and I then send not unto thee, and shew it thee;"],
    ['1Samuel','1Sa','20','013','13',"The LORD do so and much more to Jonathan: but if it please my father \emph{to} \emph{do} thee evil, then I will shew it thee, and send thee away, that thou mayest go in peace: and the LORD be with thee, as he hath been with my father."],
    ['1Samuel','1Sa','20','014','14',"And thou shalt not only while yet I live shew me the kindness of the LORD, that I die not:"],
    ['1Samuel','1Sa','20','015','15',"But \emph{also} thou shalt not cut off thy kindness from my house for ever: no, not when the LORD hath cut off the enemies of David every one from the face of the earth."],
    ['1Samuel','1Sa','20','016','16',"So Jonathan made \emph{a} \emph{covenant} with the house of David, \emph{saying}, Let the LORD even require \emph{it} at the hand of David's enemies."],
    ['1Samuel','1Sa','20','017','17',"And Jonathan caused David to swear again, because he loved him: for he loved him as he loved his own soul."],
    ['1Samuel','1Sa','20','018','18',"Then Jonathan said to David, To morrow \emph{is} the new moon: and thou shalt be missed, because thy seat will be empty."],
    ['1Samuel','1Sa','20','019','19',"And \emph{when} thou hast stayed three days, \emph{then} thou shalt go down quickly, and come to the place where thou didst hide thyself when the business was \emph{in} \emph{hand}, and shalt remain by the stone Ezel."],
    ['1Samuel','1Sa','20','020','20',"And I will shoot three arrows on the side \emph{thereof}, as though I shot at a mark."],
    ['1Samuel','1Sa','20','021','21',"And, behold, I will send a lad, \emph{saying}, Go, find out the arrows. If I expressly say unto the lad, Behold, the arrows \emph{are} on this side of thee, take them; then come thou: for \emph{there} \emph{is} peace to thee, and no hurt; \emph{as} the LORD liveth."],
    ['1Samuel','1Sa','20','022','22',"But if I say thus unto the young man, Behold, the arrows \emph{are} beyond thee; go thy way: for the LORD hath sent thee away."],
    ['1Samuel','1Sa','20','023','23',"And \emph{as} \emph{touching} the matter which thou and I have spoken of, behold, the LORD \emph{be} between thee and me for ever."],
    ['1Samuel','1Sa','20','024','24',"So David hid himself in the field: and when the new moon was come, the king sat him down to eat meat."],
    ['1Samuel','1Sa','20','025','25',"And the king sat upon his seat, as at other times, \emph{even} upon a seat by the wall: and Jonathan arose, and Abner sat by Saul's side, and David's place was empty."],
    ['1Samuel','1Sa','20','026','26',"Nevertheless Saul spake not any thing that day: for he thought, Something hath befallen him, he \emph{is} not clean; surely he \emph{is} not clean."],
    ['1Samuel','1Sa','20','027','27',"And it came to pass on the morrow, \emph{which} \emph{was} the second \emph{day} of the month, that David's place was empty: and Saul said unto Jonathan his son, Wherefore cometh not the son of Jesse to meat, neither yesterday, nor to day?"],
    ['1Samuel','1Sa','20','028','28',"And Jonathan answered Saul, David earnestly asked \emph{leave} of me \emph{to} \emph{go} to Beth-lehem:"],
    ['1Samuel','1Sa','20','029','29',"And he said, Let me go, I pray thee; for our family hath a sacrifice in the city; and my brother, he hath commanded me \emph{to} \emph{be} \emph{there}: and now, if I have found favour in thine eyes, let me get away, I pray thee, and see my brethren. Therefore he cometh not unto the king's table."],
    ['1Samuel','1Sa','20','030','30',"Then Saul's anger was kindled against Jonathan, and he said unto him, Thou son of the perverse rebellious \emph{woman}, do not I know that thou hast chosen the son of Jesse to thine own confusion, and unto the confusion of thy mother's nakedness?"],
    ['1Samuel','1Sa','20','031','31',"For as long as the son of Jesse liveth upon the ground, thou shalt not be established, nor thy kingdom. Wherefore now send and fetch him unto me, for he shall surely die."],
    ['1Samuel','1Sa','20','032','32',"And Jonathan answered Saul his father, and said unto him, Wherefore shall he be slain? what hath he done?"],
    ['1Samuel','1Sa','20','033','33',"And Saul cast a javelin at him to smite him: whereby Jonathan knew that it was determined of his father to slay David."],
    ['1Samuel','1Sa','20','034','34',"So Jonathan arose from the table in fierce anger, and did eat no meat the second day of the month: for he was grieved for David, because his father had done him shame."],
    ['1Samuel','1Sa','20','035','35',"And it came to pass in the morning, that Jonathan went out into the field at the time appointed with David, and a little lad with him."],
    ['1Samuel','1Sa','20','036','36',"And he said unto his lad, Run, find out now the arrows which I shoot. \emph{And} as the lad ran, he shot an arrow beyond him."],
    ['1Samuel','1Sa','20','037','37',"And when the lad was come to the place of the arrow which Jonathan had shot, Jonathan cried after the lad, and said, \emph{Is} not the arrow beyond thee?"],
    ['1Samuel','1Sa','20','038','38',"And Jonathan cried after the lad, Make speed, haste, stay not. And Jonathan's lad gathered up the arrows, and came to his master."],
    ['1Samuel','1Sa','20','039','39',"But the lad knew not any thing: only Jonathan and David knew the matter."],
    ['1Samuel','1Sa','20','040','40',"And Jonathan gave his artillery unto his lad, and said unto him, Go, carry \emph{them} to the city."],
    ['1Samuel','1Sa','20','041','41',"\emph{And} as soon as the lad was gone, David arose out of \emph{a} \emph{place} toward the south, and fell on his face to the ground, and bowed himself three times: and they kissed one another, and wept one with another, until David exceeded."],
    ['1Samuel','1Sa','20','042','42',"And Jonathan said to David, Go in peace, forasmuch as we have sworn both of us in the name of the LORD, saying, The LORD be between me and thee, and between my seed and thy seed for ever. And he arose and departed: and Jonathan went into the city."]
]

FirstSamuel21 = [
    ['1Samuel','1Sa','21','001','1',"Then came David to Nob to Ahimelech the priest: and Ahimelech was afraid at the meeting of David, and said unto him, Why \emph{art} thou alone, and no man with thee?"],
    ['1Samuel','1Sa','21','002','2',"And David said unto Ahimelech the priest, The king hath commanded me a business, and hath said unto me, Let no man know any thing of the business whereabout I send thee, and what I have commanded thee: and I have appointed \emph{my} servants to such and such a place."],
    ['1Samuel','1Sa','21','003','3',"Now therefore what is under thine hand? give \emph{me} five \emph{loaves} \emph{of} bread in mine hand, or what there is present."],
    ['1Samuel','1Sa','21','004','4',"And the priest answered David, and said, \emph{There} \emph{is} no common bread under mine hand, but there is hallowed bread; if the young men have kept themselves at least from women."],
    ['1Samuel','1Sa','21','005','5',"And David answered the priest, and said unto him, Of a truth women \emph{have} \emph{been} kept from us about these three days, since I came out, and the vessels of the young men are holy, and \emph{the} \emph{bread} \emph{is} in a manner common, yea, though it were sanctified this day in the vessel."],
    ['1Samuel','1Sa','21','006','6',"So the priest gave him hallowed \emph{bread}: for there was no bread there but the shewbread, that was taken from before the LORD, to put hot bread in the day when it was taken away."],
    ['1Samuel','1Sa','21','007','7',"Now a certain man of the servants of Saul \emph{was} there that day, detained before the LORD; and his name \emph{was} Doeg, an Edomite, the chiefest of the herdmen that \emph{belonged} to Saul."],
    ['1Samuel','1Sa','21','008','8',"And David said unto Ahimelech, And is there not here under thine hand spear or sword? for I have neither brought my sword nor my weapons with me, because the king's business required haste."],
    ['1Samuel','1Sa','21','009','9',"And the priest said, The sword of Goliath the Philistine, whom thou slewest in the valley of Elah, behold, it \emph{is} \emph{here} wrapped in a cloth behind the ephod: if thou wilt take that, take \emph{it}: for \emph{there} \emph{is} no other save that here. And David said, \emph{There} \emph{is} none like that; give it me."],
    ['1Samuel','1Sa','21','010','10',"And David arose, and fled that day for fear of Saul, and went to Achish the king of Gath."],
    ['1Samuel','1Sa','21','011','11',"And the servants of Achish said unto him, \emph{Is} not this David the king of the land? did they not sing one to another of him in dances, saying, Saul hath slain his thousands, and David his ten thousands?"],
    ['1Samuel','1Sa','21','012','12',"And David laid up these words in his heart, and was sore afraid of Achish the king of Gath."],
    ['1Samuel','1Sa','21','013','13',"And he changed his behaviour before them, and feigned himself mad in their hands, and scrabbled on the doors of the gate, and let his spittle fall down upon his beard."],
    ['1Samuel','1Sa','21','014','14',"Then said Achish unto his servants, Lo, ye see the man is mad: wherefore \emph{then} have ye brought him to me?"],
    ['1Samuel','1Sa','21','015','15',"Have I need of mad men, that ye have brought this \emph{fellow} to play the mad man in my presence? shall this \emph{fellow} come into my house?"]
]

FirstSamuel22 = [
    ['1Samuel','1Sam','22','001','1',"David therefore departed thence, and escaped to the cave Adullam: and when his brethren and all his father's house heard \emph{it}, they went down thither to him."],
['1Samuel','1Sam','22','002','2',"And every one \emph{that} \emph{was} in distress, and every one that \emph{was} in debt, and every one \emph{that} \emph{was} discontented, gathered themselves unto him; and he became a captain over them: and there were with him about four hundred men."],
['1Samuel','1Sam','22','003','3',"And David went thence to Mizpeh of Moab: and he said unto the king of Moab, Let my father and my mother, I pray thee, come forth, \emph{and} \emph{be} with you, till I know what God will do for me."],
['1Samuel','1Sam','22','004','4',"And he brought them before the king of Moab: and they dwelt with him all the while that David was in the hold."],
['1Samuel','1Sam','22','005','5',"And the prophet Gad said unto David, Abide not in the hold; depart, and get thee into the land of Judah. Then David departed, and came into the forest of Hareth."],
['1Samuel','1Sam','22','006','6',"When Saul heard that David was discovered, and the men that \emph{were} with him, (now Saul abode in Gibeah under a tree in Ramah, having his spear in his hand, and all his servants \emph{were} standing about him;)"],
['1Samuel','1Sam','22','007','7',"Then Saul said unto his servants that stood about him, Hear now, ye Benjamites; will the son of Jesse give every one of you fields and vineyards, \emph{and} make you all captains of thousands, and captains of hundreds;"],
['1Samuel','1Sam','22','008','8',"That all of you have conspired against me, and \emph{there} \emph{is} none that sheweth me that my son hath made a league with the son of Jesse, and \emph{there} \emph{is} none of you that is sorry for me, or sheweth unto me that my son hath stirred up my servant against me, to lie in wait, as at this day?"],
['1Samuel','1Sam','22','009','9',"Then answered Doeg the Edomite, which was set over the servants of Saul, and said, I saw the son of Jesse coming to Nob, to Ahimelech the son of Ahitub."],
['1Samuel','1Sam','22','010','10',"And he enquired of the LORD for him, and gave him victuals, and gave him the sword of Goliath the Philistine."],
['1Samuel','1Sam','22','011','11',"Then the king sent to call Ahimelech the priest, the son of Ahitub, and all his father's house, the priests that \emph{were} in Nob: and they came all of them to the king."],
['1Samuel','1Sam','22','012','12',"And Saul said, Hear now, thou son of Ahitub. And he answered, Here I \emph{am}, my lord."],
['1Samuel','1Sam','22','013','13',"And Saul said unto him, Why have ye conspired against me, thou and the son of Jesse, in that thou hast given him bread, and a sword, and hast enquired of God for him, that he should rise against me, to lie in wait, as at this day?"],
['1Samuel','1Sam','22','014','14',"Then Ahimelech answered the king, and said, And who \emph{is} \emph{so} faithful among all thy servants as David, which is the king's son in law, and goeth at thy bidding, and is honourable in thine house?"],
['1Samuel','1Sam','22','015','15',"Did I then begin to enquire of God for him? be it far from me: let not the king impute \emph{any} thing unto his servant, \emph{nor} to all the house of my father: for thy servant knew nothing of all this, less or more."],
['1Samuel','1Sam','22','016','16',"And the king said, Thou shalt surely die, Ahimelech, thou, and all thy father's house."],
['1Samuel','1Sam','22','017','17',"And the king said unto the footmen that stood about him, Turn, and slay the priests of the LORD; because their hand also \emph{is} with David, and because they knew when he fled, and did not shew it to me. But the servants of the king would not put forth their hand to fall upon the priests of the LORD."],
['1Samuel','1Sam','22','018','18',"And the king said to Doeg, Turn thou, and fall upon the priests. And Doeg the Edomite turned, and he fell upon the priests, and slew on that day fourscore and five persons that did wear a linen ephod."],
['1Samuel','1Sam','22','019','19',"And Nob, the city of the priests, smote he with the edge of the sword, both men and women, children and sucklings, and oxen, and asses, and sheep, with the edge of the sword."],
['1Samuel','1Sam','22','020','20',"And one of the sons of Ahimelech the son of Ahitub, named Abiathar, escaped, and fled after David."],
['1Samuel','1Sam','22','021','21',"And Abiathar shewed David that Saul had slain the LORD'S priests."],
['1Samuel','1Sam','22','022','22',"And David said unto Abiathar, I knew \emph{it} that day, when Doeg the Edomite \emph{was} there, that he would surely tell Saul: I have occasioned \emph{the} \emph{death} of all the persons of thy father's house."],
['1Samuel','1Sam','22','023','23',"Abide thou with me, fear not: for he that seeketh my life seeketh thy life: but with me thou \emph{shalt} \emph{be} in safeguard."]
]

FirstSamuel23 = [
    ['1Samuel','1Sam','23','001','1',"Then they told David, saying, Behold, the Philistines fight against Keilah, and they rob the threshingfloors."],
['1Samuel','1Sam','23','002','2',"Therefore David enquired of the LORD, saying, Shall I go and smite these Philistines? And the LORD said unto David, Go, and smite the Philistines, and save Keilah."],
['1Samuel','1Sam','23','003','3',"And David's men said unto him, Behold, we be afraid here in Judah: how much more then if we come to Keilah against the armies of the Philistines?"],
['1Samuel','1Sam','23','004','4',"Then David enquired of the LORD yet again. And the LORD answered him and said, Arise, go down to Keilah; for I will deliver the Philistines into thine hand."],
['1Samuel','1Sam','23','005','5',"So David and his men went to Keilah, and fought with the Philistines, and brought away their cattle, and smote them with a great slaughter. So David saved the inhabitants of Keilah."],
['1Samuel','1Sam','23','006','6',"And it came to pass, when Abiathar the son of Ahimelech fled to David to Keilah, \emph{that} he came down \emph{with} an ephod in his hand."],
['1Samuel','1Sam','23','007','7',"And it was told Saul that David was come to Keilah. And Saul said, God hath delivered him into mine hand; for he is shut in, by entering into a town that hath gates and bars."],
['1Samuel','1Sam','23','008','8',"And Saul called all the people together to war, to go down to Keilah, to besiege David and his men."],
['1Samuel','1Sam','23','009','9',"And David knew that Saul secretly practised mischief against him; and he said to Abiathar the priest, Bring hither the ephod."],
['1Samuel','1Sam','23','010','10',"Then said David, O LORD God of Israel, thy servant hath certainly heard that Saul seeketh to come to Keilah, to destroy the city for my sake."],
['1Samuel','1Sam','23','011','11',"Will the men of Keilah deliver me up into his hand? will Saul come down, as thy servant hath heard? O LORD God of Israel, I beseech thee, tell thy servant. And the LORD said, He will come down."],
['1Samuel','1Sam','23','012','12',"Then said David, Will the men of Keilah deliver me and my men into the hand of Saul? And the LORD said, They will deliver \emph{thee} up."],
['1Samuel','1Sam','23','013','13',"Then David and his men, \emph{which} \emph{were} about six hundred, arose and departed out of Keilah, and went whithersoever they could go. And it was told Saul that David was escaped from Keilah; and he forbare to go forth."],
['1Samuel','1Sam','23','014','14',"And David abode in the wilderness in strong holds, and remained in a mountain in the wilderness of Ziph. And Saul sought him every day, but God delivered him not into his hand."],
['1Samuel','1Sam','23','015','15',"And David saw that Saul was come out to seek his life: and David \emph{was} in the wilderness of Ziph in a wood."],
['1Samuel','1Sam','23','016','16',"And Jonathan Saul's son arose, and went to David into the wood, and strengthened his hand in God."],
['1Samuel','1Sam','23','017','17',"And he said unto him, Fear not: for the hand of Saul my father shall not find thee; and thou shalt be king over Israel, and I shall be next unto thee; and that also Saul my father knoweth."],
['1Samuel','1Sam','23','018','18',"And they two made a covenant before the LORD: and David abode in the wood, and Jonathan went to his house."],
['1Samuel','1Sam','23','019','19',"Then came up the Ziphites to Saul to Gibeah, saying, Doth not David hide himself with us in strong holds in the wood, in the hill of Hachilah, which \emph{is} on the south of Jeshimon?"],
['1Samuel','1Sam','23','020','20',"Now therefore, O king, come down according to all the desire of thy soul to come down; and our part \emph{shall} \emph{be} to deliver him into the king's hand."],
['1Samuel','1Sam','23','021','21',"And Saul said, Blessed \emph{be} ye of the LORD; for ye have compassion on me."],
['1Samuel','1Sam','23','022','22',"Go, I pray you, prepare yet, and know and see his place where his haunt is, \emph{and} who hath seen him there: for it is told me \emph{that} he dealeth very subtilly."],
['1Samuel','1Sam','23','023','23',"See therefore, and take knowledge of all the lurking places where he hideth himself, and come ye again to me with the certainty, and I will go with you: and it shall come to pass, if he be in the land, that I will search him out throughout all the thousands of Judah."],
['1Samuel','1Sam','23','024','24',"And they arose, and went to Ziph before Saul: but David and his men \emph{were} in the wilderness of Maon, in the plain on the south of Jeshimon."],
['1Samuel','1Sam','23','025','25',"Saul also and his men went to seek \emph{him}. And they told David: wherefore he came down into a rock, and abode in the wilderness of Maon. And when Saul heard \emph{that}, he pursued after David in the wilderness of Maon."],
['1Samuel','1Sam','23','026','26',"And Saul went on this side of the mountain, and David and his men on that side of the mountain: and David made haste to get away for fear of Saul; for Saul and his men compassed David and his men round about to take them."],
['1Samuel','1Sam','23','027','27',"But there came a messenger unto Saul, saying, Haste thee, and come; for the Philistines have invaded the land."],
['1Samuel','1Sam','23','028','28',"Wherefore Saul returned from pursuing after David, and went against the Philistines: therefore they called that place Selahammahlekoth."],
['1Samuel','1Sam','23','029','29',"And David went up from thence, and dwelt in strong holds at En-gedi."]
]

FirstSamuel24 = [
    ['1Samuel','1Sam','24','001','1',"And it came to pass, when Saul was returned from following the Philistines, that it was told him, saying, Behold, David \emph{is} in the wilderness of En-gedi."],
['1Samuel','1Sam','24','002','2',"Then Saul took three thousand chosen men out of all Israel, and went to seek David and his men upon the rocks of the wild goats."],
['1Samuel','1Sam','24','003','3',"And he came to the sheepcotes by the way, where \emph{was} a cave; and Saul went in to cover his feet: and David and his men remained in the sides of the cave."],
['1Samuel','1Sam','24','004','4',"And the men of David said unto him, Behold the day of which the LORD said unto thee, Behold, I will deliver thine enemy into thine hand, that thou mayest do to him as it shall seem good unto thee. Then David arose, and cut off the skirt of Saul's robe privily."],
['1Samuel','1Sam','24','005','5',"And it came to pass afterward, that David's heart smote him, because he had cut off Saul's skirt."],
['1Samuel','1Sam','24','006','6',"And he said unto his men, The LORD forbid that I should do this thing unto my master, the LORD'S anointed, to stretch forth mine hand against him, seeing he \emph{is} the anointed of the LORD."],
['1Samuel','1Sam','24','007','7',"So David stayed his servants with these words, and suffered them not to rise against Saul. But Saul rose up out of the cave, and went on \emph{his} way."],
['1Samuel','1Sam','24','008','8',"David also arose afterward, and went out of the cave, and cried after Saul, saying, My lord the king. And when Saul looked behind him, David stooped with his face to the earth, and bowed himself."],
['1Samuel','1Sam','24','009','9',"And David said to Saul, Wherefore hearest thou men's words, saying, Behold, David seeketh thy hurt?"],
['1Samuel','1Sam','24','010','10',"Behold, this day thine eyes have seen how that the LORD had delivered thee to day into mine hand in the cave: and \emph{some} bade \emph{me} kill thee: but \emph{mine} \emph{eye} spared thee; and I said, I will not put forth mine hand against my lord; for he \emph{is} the LORD'S anointed."],
['1Samuel','1Sam','24','011','11',"Moreover, my father, see, yea, see the skirt of thy robe in my hand: for in that I cut off the skirt of thy robe, and killed thee not, know thou and see that \emph{there} \emph{is} neither evil nor transgression in mine hand, and I have not sinned against thee; yet thou huntest my soul to take it."],
['1Samuel','1Sam','24','012','12',"The LORD judge between me and thee, and the LORD avenge me of thee: but mine hand shall not be upon thee."],
['1Samuel','1Sam','24','013','13',"As saith the proverb of the ancients, Wickedness proceedeth from the wicked: but mine hand shall not be upon thee."],
['1Samuel','1Sam','24','014','14',"After whom is the king of Israel come out? after whom dost thou pursue? after a dead dog, after a flea."],
['1Samuel','1Sam','24','015','15',"The LORD therefore be judge, and judge between me and thee, and see, and plead my cause, and deliver me out of thine hand."],
['1Samuel','1Sam','24','016','16',"And it came to pass, when David had made an end of speaking these words unto Saul, that Saul said, \emph{Is} this thy voice, my son David? And Saul lifted up his voice, and wept."],
['1Samuel','1Sam','24','017','17',"And he said to David, Thou \emph{art} more righteous than I: for thou hast rewarded me good, whereas I have rewarded thee evil."],
['1Samuel','1Sam','24','018','18',"And thou hast shewed this day how that thou hast dealt well with me: forasmuch as when the LORD had delivered me into thine hand, thou killedst me not."],
['1Samuel','1Sam','24','019','19',"For if a man find his enemy, will he let him go well away? wherefore the LORD reward thee good for that thou hast done unto me this day."],
['1Samuel','1Sam','24','020','20',"And now, behold, I know well that thou shalt surely be king, and that the kingdom of Israel shall be established in thine hand."],
['1Samuel','1Sam','24','021','21',"Swear now therefore unto me by the LORD, that thou wilt not cut off my seed after me, and that thou wilt not destroy my name out of my father's house."],
['1Samuel','1Sam','24','022','22',"And David sware unto Saul. And Saul went home; but David and his men gat them up unto the hold."]
]

FirstSamuel25 = [
    ['1Samuel','1Sam','25','001','1',"And Samuel died; and all the Israelites were gathered together, and lamented him, and buried him in his house at Ramah. And David arose, and went down to the wilderness of Paran."],
    ['1Samuel','1Sam','25','002','2',"And \emph{there} \emph{was} a man in Maon, whose possessions \emph{were} in Carmel; and the man \emph{was} very great, and he had three thousand sheep, and a thousand goats: and he was shearing his sheep in Carmel."],
    ['1Samuel','1Sam','25','003','3',"Now the name of the man \emph{was} Nabal; and the name of his wife Abigail: and \emph{she} \emph{was} a woman of good understanding, and of a beautiful countenance: but the man \emph{was} churlish and evil in his doings; and he \emph{was} of the house of Caleb."],
    ['1Samuel','1Sam','25','004','4',"And David heard in the wilderness that Nabal did shear his sheep."],
    ['1Samuel','1Sam','25','005','5',"And David sent out ten young men, and David said unto the young men, Get you up to Carmel, and go to Nabal, and greet him in my name:"],
    ['1Samuel','1Sam','25','006','6',"And thus shall ye say to him that liveth \emph{in} \emph{prosperity}, Peace \emph{be} both to thee, and peace \emph{be} to thine house, and peace \emph{be} unto all that thou hast."],
    ['1Samuel','1Sam','25','007','7',"And now I have heard that thou hast shearers: now thy shepherds which were with us, we hurt them not, neither was there ought missing unto them, all the while they were in Carmel."],
    ['1Samuel','1Sam','25','008','8',"Ask thy young men, and they will shew thee. Wherefore let the young men find favour in thine eyes: for we come in a good day: give, I pray thee, whatsoever cometh to thine hand unto thy servants, and to thy son David."],
    ['1Samuel','1Sam','25','009','9',"And when David's young men came, they spake to Nabal according to all those words in the name of David, and ceased."],
    ['1Samuel','1Sam','25','010','10',"And Nabal answered David's servants, and said, Who \emph{is} David? and who \emph{is} the son of Jesse? there be many servants now a days that break away every man from his master."],
    ['1Samuel','1Sam','25','011','11',"Shall I then take my bread, and my water, and my flesh that I have killed for my shearers, and give \emph{it} unto men, whom I know not whence they \emph{be}?"],
    ['1Samuel','1Sam','25','012','12',"So David's young men turned their way, and went again, and came and told him all those sayings."],
    ['1Samuel','1Sam','25','013','13',"And David said unto his men, Gird ye on every man his sword. And they girded on every man his sword; and David also girded on his sword: and there went up after David about four hundred men; and two hundred abode by the stuff."],
    ['1Samuel','1Sam','25','014','14',"But one of the young men told Abigail, Nabal's wife, saying, Behold, David sent messengers out of the wilderness to salute our master; and he railed on them."],
    ['1Samuel','1Sam','25','015','15',"But the men \emph{were} very good unto us, and we were not hurt, neither missed we any thing, as long as we were conversant with them, when we were in the fields:"],
    ['1Samuel','1Sam','25','016','16',"They were a wall unto us both by night and day, all the while we were with them keeping the sheep."],
    ['1Samuel','1Sam','25','017','17',"Now therefore know and consider what thou wilt do; for evil is determined against our master, and against all his household: for he \emph{is} \emph{such} a son of Belial, that \emph{a} \emph{man} cannot speak to him."],
    ['1Samuel','1Sam','25','018','18',"Then Abigail made haste, and took two hundred loaves, and two bottles of wine, and five sheep ready dressed, and five measures of parched \emph{corn}, and an hundred clusters of raisins, and two hundred cakes of figs, and laid \emph{them} on asses."],
    ['1Samuel','1Sam','25','019','19',"And she said unto her servants, Go on before me; behold, I come after you. But she told not her husband Nabal."],
    ['1Samuel','1Sam','25','020','20',"And it was \emph{so,} \emph{as} she rode on the ass, that she came down by the covert of the hill, and, behold, David and his men came down against her; and she met them."],
    ['1Samuel','1Sam','25','021','21',"Now David had said, Surely in vain have I kept all that this \emph{fellow} hath in the wilderness, so that nothing was missed of all that \emph{pertained} unto him: and he hath requited me evil for good."],
    ['1Samuel','1Sam','25','022','22',"So and more also do God unto the enemies of David, if I leave of all that \emph{pertain} to him by the morning light any that pisseth against the wall."],
    ['1Samuel','1Sam','25','023','23',"And when Abigail saw David, she hasted, and lighted off the ass, and fell before David on her face, and bowed herself to the ground,"],
    ['1Samuel','1Sam','25','024','24',"And fell at his feet, and said, Upon me, my lord, \emph{upon} me \emph{let} \emph{this} iniquity \emph{be}: and let thine handmaid, I pray thee, speak in thine audience, and hear the words of thine handmaid."],
    ['1Samuel','1Sam','25','025','25',"Let not my lord, I pray thee, regard this man of Belial, \emph{even} Nabal: for as his name \emph{is}, so \emph{is} he; Nabal \emph{is} his name, and folly \emph{is} with him: but I thine handmaid saw not the young men of my lord, whom thou didst send."],
    ['1Samuel','1Sam','25','026','26',"Now therefore, my lord, \emph{as} the LORD liveth, and \emph{as} thy soul liveth, seeing the LORD hath withholden thee from coming to \emph{shed} blood, and from avenging thyself with thine own hand, now let thine enemies, and they that seek evil to my lord, be as Nabal."],
    ['1Samuel','1Sam','25','027','27',"And now this blessing which thine handmaid hath brought unto my lord, let it even be given unto the young men that follow my lord."],
    ['1Samuel','1Sam','25','028','28',"I pray thee, forgive the trespass of thine handmaid: for the LORD will certainly make my lord a sure house; because my lord fighteth the battles of the LORD, and evil hath not been found in thee \emph{all} thy days."],
    ['1Samuel','1Sam','25','029','29',"Yet a man is risen to pursue thee, and to seek thy soul: but the soul of my lord shall be bound in the bundle of life with the LORD thy God; and the souls of thine enemies, them shall he sling out, \emph{as} \emph{out} of the middle of a sling."],
    ['1Samuel','1Sam','25','030','30',"And it shall come to pass, when the LORD shall have done to my lord according to all the good that he hath spoken concerning thee, and shall have appointed thee ruler over Israel;"],
    ['1Samuel','1Sam','25','031','31',"That this shall be no grief unto thee, nor offence of heart unto my lord, either that thou hast shed blood causeless, or that my lord hath avenged himself: but when the LORD shall have dealt well with my lord, then remember thine handmaid."],
    ['1Samuel','1Sam','25','032','32',"And David said to Abigail, Blessed \emph{be} the LORD God of Israel, which sent thee this day to meet me:"],
    ['1Samuel','1Sam','25','033','33',"And blessed \emph{be} thy advice, and blessed \emph{be} thou, which hast kept me this day from coming to \emph{shed} blood, and from avenging myself with mine own hand."],
    ['1Samuel','1Sam','25','034','34',"For in very deed, \emph{as} the LORD God of Israel liveth, which hath kept me back from hurting thee, except thou hadst hasted and come to meet me, surely there had not been left unto Nabal by the morning light any that pisseth against the wall."],
    ['1Samuel','1Sam','25','035','35',"So David received of her hand \emph{that} which she had brought him, and said unto her, Go up in peace to thine house; see, I have hearkened to thy voice, and have accepted thy person."],
    ['1Samuel','1Sam','25','036','36',"And Abigail came to Nabal; and, behold, he held a feast in his house, like the feast of a king; and Nabal's heart \emph{was} merry within him, for he \emph{was} very drunken: wherefore she told him nothing, less or more, until the morning light."],
    ['1Samuel','1Sam','25','037','37',"But it came to pass in the morning, when the wine was gone out of Nabal, and his wife had told him these things, that his heart died within him, and he became \emph{as} a stone."],
    ['1Samuel','1Sam','25','038','38',"And it came to pass about ten days \emph{after}, that the LORD smote Nabal, that he died."],
    ['1Samuel','1Sam','25','039','39',"And when David heard that Nabal was dead, he said, Blessed \emph{be} the LORD, that hath pleaded the cause of my reproach from the hand of Nabal, and hath kept his servant from evil: for the LORD hath returned the wickedness of Nabal upon his own head. And David sent and communed with Abigail, to take her to him to wife."],
    ['1Samuel','1Sam','25','040','40',"And when the servants of David were come to Abigail to Carmel, they spake unto her, saying, David sent us unto thee, to take thee to him to wife."],
    ['1Samuel','1Sam','25','041','41',"And she arose, and bowed herself on \emph{her} face to the earth, and said, Behold, \emph{let} thine handmaid \emph{be} a servant to wash the feet of the servants of my lord."],
    ['1Samuel','1Sam','25','042','42',"And Abigail hasted, and arose, and rode upon an ass, with five damsels of hers that went after her; and she went after the messengers of David, and became his wife."],
    ['1Samuel','1Sam','25','043','43',"David also took Ahinoam of Jezreel; and they were also both of them his wives."],
    ['1Samuel','1Sam','25','044','44',"But Saul had given Michal his daughter, David's wife, to Phalti the son of Laish, which \emph{was} of Gallim."]
]

FirstSamuel26 = [
    ['1Samuel','1Sam','26','001','1',"And the Ziphites came unto Saul to Gibeah, saying, Doth not David hide himself in the hill of Hachilah, \emph{which} \emph{is} before Jeshimon?"],
['1Samuel','1Sam','26','002','2',"Then Saul arose, and went down to the wilderness of Ziph, having three thousand chosen men of Israel with him, to seek David in the wilderness of Ziph."],
['1Samuel','1Sam','26','003','3',"And Saul pitched in the hill of Hachilah, which \emph{is} before Jeshimon, by the way. But David abode in the wilderness, and he saw that Saul came after him into the wilderness."],
['1Samuel','1Sam','26','004','4',"David therefore sent out spies, and understood that Saul was come in very deed."],
['1Samuel','1Sam','26','005','5',"And David arose, and came to the place where Saul had pitched: and David beheld the place where Saul lay, and Abner the son of Ner, the captain of his host: and Saul lay in the trench, and the people pitched round about him."],
['1Samuel','1Sam','26','006','6',"Then answered David and said to Ahimelech the Hittite, and to Abishai the son of Zeruiah, brother to Joab, saying, Who will go down with me to Saul to the camp? And Abishai said, I will go down with thee."],
['1Samuel','1Sam','26','007','7',"So David and Abishai came to the people by night: and, behold, Saul lay sleeping within the trench, and his spear stuck in the ground at his bolster: but Abner and the people lay round about him."],
['1Samuel','1Sam','26','008','8',"Then said Abishai to David, God hath delivered thine enemy into thine hand this day: now therefore let me smite him, I pray thee, with the spear even to the earth at once, and I will not \emph{smite} him the second time."],
['1Samuel','1Sam','26','009','9',"And David said to Abishai, Destroy him not: for who can stretch forth his hand against the LORD'S anointed, and be guiltless?"],
['1Samuel','1Sam','26','010','10',"David said furthermore, \emph{As} the LORD liveth, the LORD shall smite him; or his day shall come to die; or he shall descend into battle, and perish."],
['1Samuel','1Sam','26','011','11',"The LORD forbid that I should stretch forth mine hand against the LORD'S anointed: but, I pray thee, take thou now the spear that \emph{is} at his bolster, and the cruse of water, and let us go."],
['1Samuel','1Sam','26','012','12',"So David took the spear and the cruse of water from Saul's bolster; and they gat them away, and no man saw \emph{it}, nor knew \emph{it}, neither awaked: for they \emph{were} all asleep; because a deep sleep from the LORD was fallen upon them."],
['1Samuel','1Sam','26','013','13',"Then David went over to the other side, and stood on the top of an hill afar off; a great space \emph{being} between them:"],
['1Samuel','1Sam','26','014','14',"And David cried to the people, and to Abner the son of Ner, saying, Answerest thou not, Abner? Then Abner answered and said, Who \emph{art} thou \emph{that} criest to the king?"],
['1Samuel','1Sam','26','015','15',"And David said to Abner, \emph{Art} not thou a \emph{valiant} man? and who \emph{is} like to thee in Israel? wherefore then hast thou not kept thy lord the king? for there came one of the people in to destroy the king thy lord."],
['1Samuel','1Sam','26','016','16',"This thing \emph{is} not good that thou hast done. \emph{As} the LORD liveth, ye \emph{are} worthy to die, because ye have not kept your master, the LORD'S anointed. And now see where the king's spear \emph{is}, and the cruse of water that \emph{was} at his bolster."],
['1Samuel','1Sam','26','017','17',"And Saul knew David's voice, and said, \emph{Is} this thy voice, my son David? And David said, \emph{It} \emph{is} my voice, my lord, O king."],
['1Samuel','1Sam','26','018','18',"And he said, Wherefore doth my lord thus pursue after his servant? for what have I done? or what evil \emph{is} in mine hand?"],
['1Samuel','1Sam','26','019','19',"Now therefore, I pray thee, let my lord the king hear the words of his servant. If the LORD have stirred thee up against me, let him accept an offering: but if \emph{they} \emph{be} the children of men, cursed \emph{be} they before the LORD; for they have driven me out this day from abiding in the inheritance of the LORD, saying, Go, serve other gods."],
['1Samuel','1Sam','26','020','20',"Now therefore, let not my blood fall to the earth before the face of the LORD: for the king of Israel is come out to seek a flea, as when one doth hunt a partridge in the mountains."],
['1Samuel','1Sam','26','021','21',"Then said Saul, I have sinned: return, my son David: for I will no more do thee harm, because my soul was precious in thine eyes this day: behold, I have played the fool, and have erred exceedingly."],
['1Samuel','1Sam','26','022','22',"And David answered and said, Behold the king's spear! and let one of the young men come over and fetch it."],
['1Samuel','1Sam','26','023','23',"The LORD render to every man his righteousness and his faithfulness: for the LORD delivered thee into \emph{my} hand to day, but I would not stretch forth mine hand against the LORD'S anointed."],
['1Samuel','1Sam','26','024','24',"And, behold, as thy life was much set by this day in mine eyes, so let my life be much set by in the eyes of the LORD, and let him deliver me out of all tribulation."],
['1Samuel','1Sam','26','025','25',"Then Saul said to David, Blessed \emph{be} thou, my son David: thou shalt both do great \emph{things}, and also shalt still prevail. So David went on his way, and Saul returned to his place."]
]

FirstSamuel27 = [
    ['1Samuel','1Sam','27','001','1',"And David said in his heart, I shall now perish one day by the hand of Saul: \emph{there} \emph{is} nothing better for me than that I should speedily escape into the land of the Philistines; and Saul shall despair of me, to seek me any more in any coast of Israel: so shall I escape out of his hand."],
    ['1Samuel','1Sam','27','002','2',"And David arose, and he passed over with the six hundred men that \emph{were} with him unto Achish, the son of Maoch, king of Gath."],
    ['1Samuel','1Sam','27','003','3',"And David dwelt with Achish at Gath, he and his men, every man with his household, \emph{even} David with his two wives, Ahinoam the Jezreelitess, and Abigail the Carmelitess, Nabal's wife."],
    ['1Samuel','1Sam','27','004','4',"And it was told Saul that David was fled to Gath: and he sought no more again for him."],
    ['1Samuel','1Sam','27','005','5',"And David said unto Achish, If I have now found grace in thine eyes, let them give me a place in some town in the country, that I may dwell there: for why should thy servant dwell in the royal city with thee?"],
    ['1Samuel','1Sam','27','006','6',"Then Achish gave him Ziklag that day: wherefore Ziklag pertaineth unto the kings of Judah unto this day."],
    ['1Samuel','1Sam','27','007','7',"And the time that David dwelt in the country of the Philistines was a full year and four months."],
    ['1Samuel','1Sam','27','008','8',"And David and his men went up, and invaded the Geshurites, and the Gezrites, and the Amalekites: for those \emph{nations} \emph{were} of old the inhabitants of the land, as thou goest to Shur, even unto the land of Egypt."],
    ['1Samuel','1Sam','27','009','9',"And David smote the land, and left neither man nor woman alive, and took away the sheep, and the oxen, and the asses, and the camels, and the apparel, and returned, and came to Achish."],
    ['1Samuel','1Sam','27','010','10',"And Achish said, Whither have ye made a road to day? And David said, Against the south of Judah, and against the south of the Jerahmeelites, and against the south of the Kenites."],
    ['1Samuel','1Sam','27','011','11',"And David saved neither man nor woman alive, to bring \emph{tidings} to Gath, saying, Lest they should tell on us, saying, So did David, and so \emph{will} \emph{be} his manner all the while he dwelleth in the country of the Philistines."],
    ['1Samuel','1Sam','27','012','12',"And Achish believed David, saying, He hath made his people Israel utterly to abhor him; therefore he shall be my servant for ever."]
]

FirstSamuel28 = [
        ['1Samuel','1Sam','28','001','1',"And it came to pass in those days, that the Philistines gathered their armies together for warfare, to fight with Israel. And Achish said unto David, Know thou assuredly, that thou shalt go out with me to battle, thou and thy men."],
['1Samuel','1Sam','28','002','2',"And David said to Achish, Surely thou shalt know what thy servant can do. And Achish said to David, Therefore will I make thee keeper of mine head for ever."],
['1Samuel','1Sam','28','003','3',"Now Samuel was dead, and all Israel had lamented him, and buried him in Ramah, even in his own city. And Saul had put away those that had familiar spirits, and the wizards, out of the land."],
['1Samuel','1Sam','28','004','4',"And the Philistines gathered themselves together, and came and pitched in Shunem: and Saul gathered all Israel together, and they pitched in Gilboa."],
['1Samuel','1Sam','28','005','5',"And when Saul saw the host of the Philistines, he was afraid, and his heart greatly trembled."],
['1Samuel','1Sam','28','006','6',"And when Saul enquired of the LORD, the LORD answered him not, neither by dreams, nor by Urim, nor by prophets."],
['1Samuel','1Sam','28','007','7',"Then said Saul unto his servants, Seek me a woman that hath a familiar spirit, that I may go to her, and enquire of her. And his servants said to him, Behold, \emph{there} \emph{is} a woman that hath a familiar spirit at En-dor."],
['1Samuel','1Sam','28','008','8',"And Saul disguised himself, and put on other raiment, and he went, and two men with him, and they came to the woman by night: and he said, I pray thee, divine unto me by the familiar spirit, and bring me \emph{him} up, whom I shall name unto thee."],
['1Samuel','1Sam','28','009','9',"And the woman said unto him, Behold, thou knowest what Saul hath done, how he hath cut off those that have familiar spirits, and the wizards, out of the land: wherefore then layest thou a snare for my life, to cause me to die?"],
['1Samuel','1Sam','28','010','10',"And Saul sware to her by the LORD, saying, \emph{As} the LORD liveth, there shall no punishment happen to thee for this thing."],
['1Samuel','1Sam','28','011','11',"Then said the woman, Whom shall I bring up unto thee? And he said, Bring me up Samuel."],
['1Samuel','1Sam','28','012','12',"And when the woman saw Samuel, she cried with a loud voice: and the woman spake to Saul, saying, Why hast thou deceived me? for thou \emph{art} Saul."],
['1Samuel','1Sam','28','013','13',"And the king said unto her, Be not afraid: for what sawest thou? And the woman said unto Saul, I saw gods ascending out of the earth."],
['1Samuel','1Sam','28','014','14',"And he said unto her, What form \emph{is} he of? And she said, An old man cometh up; and he \emph{is} covered with a mantle. And Saul perceived that it \emph{was} Samuel, and he stooped with \emph{his} face to the ground, and bowed himself."],
['1Samuel','1Sam','28','015','15',"And Samuel said to Saul, Why hast thou disquieted me, to bring me up? And Saul answered, I am sore distressed; for the Philistines make war against me, and God is departed from me, and answereth me no more, neither by prophets, nor by dreams: therefore I have called thee, that thou mayest make known unto me what I shall do."],
['1Samuel','1Sam','28','016','16',"Then said Samuel, Wherefore then dost thou ask of me, seeing the LORD is departed from thee, and is become thine enemy?"],
['1Samuel','1Sam','28','017','17',"And the LORD hath done to him, as he spake by me: for the LORD hath rent the kingdom out of thine hand, and given it to thy neighbour, \emph{even} to David:"],
['1Samuel','1Sam','28','018','18',"Because thou obeyedst not the voice of the LORD, nor executedst his fierce wrath upon Amalek, therefore hath the LORD done this thing unto thee this day."],
['1Samuel','1Sam','28','019','19',"Moreover the LORD will also deliver Israel with thee into the hand of the Philistines: and to morrow \emph{shalt} thou and thy sons \emph{be} with me: the LORD also shall deliver the host of Israel into the hand of the Philistines."],
['1Samuel','1Sam','28','020','20',"Then Saul fell straightway all along on the earth, and was sore afraid, because of the words of Samuel: and there was no strength in him; for he had eaten no bread all the day, nor all the night."],
['1Samuel','1Sam','28','021','21',"And the woman came unto Saul, and saw that he was sore troubled, and said unto him, Behold, thine handmaid hath obeyed thy voice, and I have put my life in my hand, and have hearkened unto thy words which thou spakest unto me."],
['1Samuel','1Sam','28','022','22',"Now therefore, I pray thee, hearken thou also unto the voice of thine handmaid, and let me set a morsel of bread before thee; and eat, that thou mayest have strength, when thou goest on thy way."],
['1Samuel','1Sam','28','023','23',"But he refused, and said, I will not eat. But his servants, together with the woman, compelled him; and he hearkened unto their voice. So he arose from the earth, and sat upon the bed."],
['1Samuel','1Sam','28','024','24',"And the woman had a fat calf in the house; and she hasted, and killed it, and took flour, and kneaded \emph{it}, and did bake unleavened bread thereof:"],
['1Samuel','1Sam','28','025','25',"And she brought \emph{it} before Saul, and before his servants; and they did eat. Then they rose up, and went away that night."]
]

FirstSamuel29 = [
    ['1Samuel','1Sam','29','001','1',"Now the Philistines gathered together all their armies to Aphek: and the Israelites pitched by a fountain which \emph{is} in Jezreel."],
['1Samuel','1Sam','29','002','2',"And the lords of the Philistines passed on by hundreds, and by thousands: but David and his men passed on in the rereward with Achish."],
['1Samuel','1Sam','29','003','3',"Then said the princes of the Philistines, What \emph{do} these Hebrews \emph{here}? And Achish said unto the princes of the Philistines, \emph{Is} not this David, the servant of Saul the king of Israel, which hath been with me these days, or these years, and I have found no fault in him since he fell \emph{unto} \emph{me} unto this day?"],
['1Samuel','1Sam','29','004','4',"And the princes of the Philistines were wroth with him; and the princes of the Philistines said unto him, Make this fellow return, that he may go again to his place which thou hast appointed him, and let him not go down with us to battle, lest in the battle he be an adversary to us: for wherewith should he reconcile himself unto his master? \emph{should} \emph{it} not \emph{be} with the heads of these men?"],
['1Samuel','1Sam','29','005','5',"\emph{Is} not this David, of whom they sang one to another in dances, saying, Saul slew his thousands, and David his ten thousands?"],
['1Samuel','1Sam','29','006','6',"Then Achish called David, and said unto him, Surely, \emph{as} the LORD liveth, thou hast been upright, and thy going out and thy coming in with me in the host \emph{is} good in my sight: for I have not found evil in thee since the day of thy coming unto me unto this day: nevertheless the lords favour thee not."],
['1Samuel','1Sam','29','007','7',"Wherefore now return, and go in peace, that thou displease not the lords of the Philistines."],
['1Samuel','1Sam','29','008','8',"And David said unto Achish, But what have I done? and what hast thou found in thy servant so long as I have been with thee unto this day, that I may not go fight against the enemies of my lord the king?"],
['1Samuel','1Sam','29','009','9',"And Achish answered and said to David, I know that thou \emph{art} good in my sight, as an angel of God: notwithstanding the princes of the Philistines have said, He shall not go up with us to the battle."],
['1Samuel','1Sam','29','010','10',"Wherefore now rise up early in the morning with thy master's servants that are come with thee: and as soon as ye be up early in the morning, and have light, depart."],
['1Samuel','1Sam','29','011','11',"So David and his men rose up early to depart in the morning, to return into the land of the Philistines. And the Philistines went up to Jezreel."]
]

FirstSamuel30 = [
    ['1Samuel','1Sam','30','001','1',"And it came to pass, when David and his men were come to Ziklag on the third day, that the Amalekites had invaded the south, and Ziklag, and smitten Ziklag, and burned it with fire;"],
    ['1Samuel','1Sam','30','002','2',"And had taken the women captives, that \emph{were} therein: they slew not any, either great or small, but carried \emph{them} away, and went on their way."],
    ['1Samuel','1Sam','30','003','3',"So David and his men came to the city, and, behold, \emph{it} \emph{was} burned with fire; and their wives, and their sons, and their daughters, were taken captives."],
    ['1Samuel','1Sam','30','004','4',"Then David and the people that \emph{were} with him lifted up their voice and wept, until they had no more power to weep."],
    ['1Samuel','1Sam','30','005','5',"And David's two wives were taken captives, Ahinoam the Jezreelitess, and Abigail the wife of Nabal the Carmelite."],
    ['1Samuel','1Sam','30','006','6',"And David was greatly distressed; for the people spake of stoning him, because the soul of all the people was grieved, every man for his sons and for his daughters: but David encouraged himself in the LORD his God."],
    ['1Samuel','1Sam','30','007','7',"And David said to Abiathar the priest, Ahimelech's son, I pray thee, bring me hither the ephod. And Abiathar brought thither the ephod to David."],
    ['1Samuel','1Sam','30','008','8',"And David enquired at the LORD, saying, Shall I pursue after this troop? shall I overtake them? And he answered him, Pursue: for thou shalt surely overtake \emph{them}, and without fail recover \emph{all}."],
    ['1Samuel','1Sam','30','009','9',"So David went, he and the six hundred men that \emph{were} with him, and came to the brook Besor, where those that were left behind stayed."],
    ['1Samuel','1Sam','30','010','10',"But David pursued, he and four hundred men: for two hundred abode behind, which were so faint that they could not go over the brook Besor."],
    ['1Samuel','1Sam','30','011','11',"And they found an Egyptian in the field, and brought him to David, and gave him bread, and he did eat; and they made him drink water;"],
    ['1Samuel','1Sam','30','012','12',"And they gave him a piece of a cake of figs, and two clusters of raisins: and when he had eaten, his spirit came again to him: for he had eaten no bread, nor drunk \emph{any} water, three days and three nights."],
    ['1Samuel','1Sam','30','013','13',"And David said unto him, To whom \emph{belongest} thou? and whence \emph{art} thou? And he said, I \emph{am} a young man of Egypt, servant to an Amalekite; and my master left me, because three days agone I fell sick."],
    ['1Samuel','1Sam','30','014','14',"We made an invasion \emph{upon} the south of the Cherethites, and upon \emph{the} \emph{coast} which \emph{belongeth} to Judah, and upon the south of Caleb; and we burned Ziklag with fire."],
    ['1Samuel','1Sam','30','015','15',"And David said to him, Canst thou bring me down to this company? And he said, Swear unto me by God, that thou wilt neither kill me, nor deliver me into the hands of my master, and I will bring thee down to this company."],
    ['1Samuel','1Sam','30','016','16',"And when he had brought him down, behold, \emph{they} \emph{were} spread abroad upon all the earth, eating and drinking, and dancing, because of all the great spoil that they had taken out of the land of the Philistines, and out of the land of Judah."],
    ['1Samuel','1Sam','30','017','17',"And David smote them from the twilight even unto the evening of the next day: and there escaped not a man of them, save four hundred young men, which rode upon camels, and fled."],
    ['1Samuel','1Sam','30','018','18',"And David recovered all that the Amalekites had carried away: and David rescued his two wives."],
    ['1Samuel','1Sam','30','019','19',"And there was nothing lacking to them, neither small nor great, neither sons nor daughters, neither spoil, nor any \emph{thing} that they had taken to them: David recovered all."],
    ['1Samuel','1Sam','30','020','20',"And David took all the flocks and the herds, \emph{which} they drave before those \emph{other} cattle, and said, This \emph{is} David's spoil."],
    ['1Samuel','1Sam','30','021','21',"And David came to the two hundred men, which were so faint that they could not follow David, whom they had made also to abide at the brook Besor: and they went forth to meet David, and to meet the people that \emph{were} with him: and when David came near to the people, he saluted them."],
    ['1Samuel','1Sam','30','022','22',"Then answered all the wicked men and \emph{men} of Belial, of those that went with David, and said, Because they went not with us, we will not give them \emph{ought} of the spoil that we have recovered, save to every man his wife and his children, that they may lead \emph{them} away, and depart."],
    ['1Samuel','1Sam','30','023','23',"Then said David, Ye shall not do so, my brethren, with that which the LORD hath given us, who hath preserved us, and delivered the company that came against us into our hand."],
    ['1Samuel','1Sam','30','024','24',"For who will hearken unto you in this matter? but as his part \emph{is} that goeth down to the battle, so \emph{shall} his part \emph{be} that tarrieth by the stuff: they shall part alike."],
    ['1Samuel','1Sam','30','025','25',"And it was \emph{so} from that day forward, that he made it a statute and an ordinance for Israel unto this day."],
    ['1Samuel','1Sam','30','026','26',"And when David came to Ziklag, he sent of the spoil unto the elders of Judah, \emph{even} to his friends, saying, Behold a present for you of the spoil of the enemies of the LORD;"],
    ['1Samuel','1Sam','30','027','27',"To \emph{them} which \emph{were} in Beth-el, and to \emph{them} which \emph{were} in south Ramoth, and to \emph{them} which \emph{were} in Jattir,"],
    ['1Samuel','1Sam','30','028','28',"And to \emph{them} which \emph{were} in Aroer, and to \emph{them} which \emph{were} in Siphmoth, and to \emph{them} which \emph{were} in Eshtemoa,"],
    ['1Samuel','1Sam','30','029','29',"And to \emph{them} which \emph{were} in Rachal, and to \emph{them} which \emph{were} in the cities of the Jerahmeelites, and to \emph{them} which \emph{were} in the cities of the Kenites,"],
    ['1Samuel','1Sam','30','030','30',"And to \emph{them} which \emph{were} in Hormah, and to \emph{them} which \emph{were} in Chorashan, and to \emph{them} which \emph{were} in Athach,"],
    ['1Samuel','1Sam','30','031','31',"And to \emph{them} which \emph{were} in Hebron, and to all the places where David himself and his men were wont to haunt."]
]

FirstSamuel31 = [
    ['1Samuel','1Sam','31','001', '1',"Now the Philistines fought against Israel: and the men of Israel fled from before the Philistines, and fell down slain in mount Gilboa."],
    ['1Samuel','1Sam','31','002', '2',"And the Philistines followed hard upon Saul and upon his sons; and the Philistines slew Jonathan, and Abinadab, and Malchi-shua, Saul's sons."],
    ['1Samuel','1Sam','31','003', '3',"And the battle went sore against Saul, and the archers hit him; and he was sore wounded of the archers."],
    ['1Samuel','1Sam','31','004', '4',"Then said Saul unto his armourbearer, Draw thy sword, and thrust me through therewith; lest these uncircumcised come and thrust me through, and abuse me. But his armourbearer would not; for he was sore afraid. Therefore Saul took a sword, and fell upon it."],
    ['1Samuel','1Sam','31','005', '5',"And when his armourbearer saw that Saul was dead, he fell likewise upon his sword, and died with him."],
    ['1Samuel','1Sam','31','006', '6',"So Saul died, and his three sons, and his armourbearer, and all his men, that same day together."],
    ['1Samuel','1Sam','31','007', '7',"And when the men of Israel that \emph{were} on the other side of the valley, and \emph{they} that \emph{were} on the other side Jordan, saw that the men of Israel fled, and that Saul and his sons were dead, they forsook the cities, and fled; and the Philistines came and dwelt in them."],
    ['1Samuel','1Sam','31','008', '8',"And it came to pass on the morrow, when the Philistines came to strip the slain, that they found Saul and his three sons fallen in mount Gilboa."],
    ['1Samuel','1Sam','31','009', '9',"And they cut off his head, and stripped off his armour, and sent into the land of the Philistines round about, to publish \emph{it} \emph{in} the house of their idols, and among the people."],
    ['1Samuel','1Sam','31','010','10',"And they put his armour in the house of Ashtaroth: and they fastened his body to the wall of Beth-shan."],
    ['1Samuel','1Sam','31','011','11',"And when the inhabitants of Jabesh-gilead heard of that which the Philistines had done to Saul;"],
    ['1Samuel','1Sam','31','012','12',"All the valiant men arose, and went all night, and took the body of Saul and the bodies of his sons from the wall of Beth-shan, and came to Jabesh, and burnt them there."],
    ['1Samuel','1Sam','31','013','13',"And they took their bones, and buried \emph{them} under a tree at Jabesh, and fasted seven days."]
]

def main():
    ProcessVerses('FirstSamuel15',FirstSamuel15)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{Samuel also said unto Saul, The LORD sent me to anoint thee \emph{to} \emph{be} king over his people, over Israel: now therefore hearken thou unto the voice of the words of the LORD.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{Thus saith the LORD of hosts, I remember \emph{that} which Amalek did to Israel, how he laid \emph{wait} for him in the way, when he came up from Egypt.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{Now go and smite Amalek, and utterly destroy all that they have, and spare them not; but slay both man and woman, infant and suckling, ox and sheep, camel and ass.}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And Saul gathered the people together, and numbered them in Telaim, two hundred thousand footmen, and ten thousand men of Judah.}
[5] \textcolor[cmyk]{0.99998,1,0,0}{And Saul 

Israel & 10 \\ \hline
It & 1 \\ \hline
Judah & 1 \\ \hline
Kenites & 2 \\ \hline
LORD & 28 \\ \hline
Now & 2 \\ \hline
Ramah & 1 \\ \hline
Samuel & 22 \\ \hline
Saul & 23 \\ \hline
Say & 1 \\ \hline
Shur & 1 \\ \hline
So & 2 \\ \hline
Stay & 1 \\ \hline
Strength & 1 \\ \hline
Surely & 1 \\ \hline
Telaim & 1 \\ \hline
The & 2 \\ \hline
Then & 5 \\ \hline
They & 1 \\ \hline
Thus & 1 \\ \hline
What & 1 \\ \hline
When & 1 \\ \hline
Wherefore & 1 \\ \hline
Yea & 1 \\ \hline
\emph{and} & 1 \\ \hline
\emph{as} & 3 \\ \hline
\emph{being} & 1 \\ \hline
\emph{be} & 3 \\ \hline
\emph{great} & 1 \\ \hline
\emph{is} & 6 \\ \hline
\emph{made} & 1 \\ \hline
\emph{meaneth} & 1 \\ \hline
\emph{that} & 4 \\ \hline
\emph{to} & 2 \\ \hline
\emph{until} & 1 \\ \hline
\emph{wait} & 1 \\ \hline
\emph{wast} & 2 \\ \hline
\emph{was} & 2 \\ \hline
\emph{yet} & 1 \\ \hline
a & 5 \\ \hline
about & 2 \\ \hline
after & 1 \\ \hline
again & 3 \\ \hline
against & 2 \\ \hline
alive & 1 \\ \hline
all & 5 \\ \hline
also 